<a href="https://colab.research.google.com/github/EleonoraBartolomucci/Fairness/blob/master/PipelineAmazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install python-dateutil lxml requests selectorlib gender_guesser

     |████████████████████████████████| 389kB 5.2MB/s 


In [ ]:
#IMPORT, FUNZIONI PRINCIPALI E METODI DI SUPPORTO
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
#from clarifai.rest import ClarifaiApp
#from clarifai.rest import ApiError
import ast
import numpy as np
import time
import json
import csv
import random
import pandas as pd
import requests
from lxml import html
import shutil
import itertools
import os
import pickle
import datetime
import math
import io
import math
from google.colab import files
from selectorlib import Extractor
import requests 
from time import sleep
from dateutil import parser as dateparser
import gender_guesser.detector as gender

import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import statistics as st
from scipy import stats
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import SpectralClustering
from sklearn.cluster import KMeans

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

# CONSTANTS
business_headers = ['index', 'business_id', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'is_open', 'attributes', 'categories', 'hours']

# AUTHENTICATE IN GOOGLE DRIVE
def authenticate():
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  return drive
drive = authenticate()


def create_folder_in_drive(gdrive, folder_name, parent_folder_id):
  folder_metadata = {'title': folder_name,'mimeType': 'application/vnd.google-apps.folder',
                    'parents': [{"kind": "drive#fileLink", "id": parent_folder_id}]
                    }
  folder = gdrive.CreateFile(folder_metadata)
  folder.Upload()
  print(folder)
  # Return folder informations
  print('title: %s, id: %s' % (folder['title'], folder['id']))
  return folder['id']


def drop_unnamed(df):
  cols = [c for c in df.columns if c.lower()[:7] != 'unnamed']
  return df[cols]


def upload_file(filename, folder_id):
  drive = authenticate()
  fileList = drive.ListFile({'q': "'" + folder_id + "' in parents and trashed=false"}).GetList()
  drive_file = drive.CreateFile({'title': filename, 'parents': [{'id': folder_id}]})
  # Check if file already exists in Google Drive (prevents duplicates)
  for file in fileList:
      if file['title'] == filename:  # The file already exists, then overwrite it
          fileID = file['id']
          drive_file = drive.CreateFile({'id': fileID, 'title': filename, 'parents': [{'id': folder_id}]})
  # Upload user picture on Google Drive
  drive_file.SetContentFile(filename)  # path of local file content
  drive_file.Upload()  # Upload the file.
  return drive_file['id']

def read_json(json_path):
  data = []
  with open(json_path, "r") as my_file: 
    for line in my_file:
      line_json = json.loads(line)
      data.append(line_json)
  return data


# PARSE JSON IN CSV
def json2csv(csv_path, json_path):
  data = read_json(json_path)
  df = pd.DataFrame(data)
  df.to_csv(csv_path)


def drop_unnamed(df):
  cols = [c for c in df.columns if c.lower()[:7] != 'unnamed']
  return df[cols]

# JOIN USER FROM AMAZON WITH USER FROM DATASET AND PRINT LOST USERS
def filter_user_from_dataset(df, users):
  df = df[['position','user_id','date','location']] # Tolgo alcune colonne perché prendo quelle del dataset
  df_merged = df.merge(users, on='user_id')
  total_review = df['position'].max()
  print('Utenti persi: totali ' + str(total_review) + ' - utenti nel dataset ' +
        str(len(df_merged.index)) + ' = ' + str(total_review - len(df_merged.index)))
  df_merged['position'] = df_merged.index + 1
  df_merged = drop_unnamed(df_merged)
  return df_merged

def compute_groups_percents(df_groups, N_of_groups):
  total = len(df_groups.index)
  percents = []
  i = 0
  while i < N_of_groups:
    current_length = len(df_groups[df_groups['group_id'] == i].index)
    percents.append((current_length/total)*100)
    i = i + 1
  print(percents)
  return percents

# ADD REVIEW INFO IN RANKING DATAFRAME
#def integrate_review_info(df, reviews, business_id):
  #business_reviews = reviews[reviews['business_id'] == business_id]
  #df_merged = business_reviews.merge(df, on='user_id')
  #del df_merged['date_y']  # it's date from amazon website (not updated in dataset)
  #df_merged = df_merged.rename(columns={'date_x':'date'})
  ## drop duplicates reviews from same user
  #df_merged = df_merged.sort_values('date').drop_duplicates('user_id',keep='last')
  #df_merged = df_merged.sort_values(by=['position']).reset_index(drop=True)
  #print('Review perse: ', len(df.index) - len(df_merged.index))
  #df_merged['position'] = df_merged.index + 1
  #df_merged = drop_unnamed(df_merged)
  #df_merged = df_merged[['position', 'user_id', 'review_id', 'date', 'name', 'location', 'text', 'fans', 'average_stars', 'review_count', 'business_id']]
  
  #return df_merged

def reorder_user_data(df):
  #df['review_count'] = np.NaN
  #df['location'] = np.NaN
  #df['useful_votes'] = np.NaN
  #df['position'] = np.NaN
  #df['text'] = ""
  for i, user in df.iterrows():
    print(i)
    user_data = user['user']
    user_data_converted = ast.literal_eval(user_data)
    
    #df.loc[i, 'review_count'] = i+1
    #df.loc[i, 'friend_count'] = user_data_converted["friendCount"]
    #df.loc[i, 'location'] = user_data_converted['country']
    #df.loc[i, 'photo_count'] = user_data_converted['photoCount']
    comment_data = user['comment']
    #comment_data_converted = ast.literal_eval(comment_data)
    df.loc[i, 'text'] = comment_data[0]
    feedback_data = user['feedback']
    feedback_data_converted = ast.literal_eval(feedback_data)
    counts_data = feedback_data_converted['counts']
    #counts_data_converted = ast.literal_eval(counts_data)
    df.loc[i, 'useful_votes'] = int(counts_data['useful'])
    df.loc[i, 'position'] = int(user['position'])
  print('REORDER-----------------------------------------')
  print(df)
  df = drop_unnamed(df)
  return df

def generate_dummies(df, text_attribute_list):
  dummy_columns = []
  for attr in text_attribute_list:
    gender_dummies = pd.get_dummies(df[attr])
    dummy_columns = dummy_columns + list(gender_dummies.columns)
    df = pd.merge(df, gender_dummies, how="left",left_index=True, right_index=True)
    
    # drop all the unnamed columns
    df = drop_unnamed(df)
  return df, dummy_columns

def create_vectors_yelp(df_users, business_id, list_of_attributes, method, destination):
  vectors = df_users[['user_id']]#, 'review_count', 'friend_count']]#, 'location', 'photo_count']]
  # DEMOGRAPHICS
  vectors = get_demographics(vectors, business_id)
  # SENTIMENT
  #vectors = get_sentiment(vectors, business_id)

  print(vectors.columns)
  # UPLOAD VECTORS IN DRIVE
  vectors.to_csv('user_vectors_' + business_id + '.csv')
  upload_file('user_vectors_' + business_id + '.csv',destination)
  return vectors

def get_sentiment(vectors, id):
  df_sentiment = pd.read_csv('sentiment_' + id + '.csv')
  new_vectors = pd.merge(vectors, df_sentiment, on='user_id', how='left')
  new_vectors = drop_unnamed(new_vectors)
  return new_vectors


def get_demographics(vectors, id):
  print(id)
  demographics_file_id = gtree.loc[gtree['asin']==id, 'gfolder_clarifai'].tolist()[0]
  download = drive.CreateFile({'id': demographics_file_id})
  download.GetContentFile('demographics_'  + id + '.csv')
  df_demographics = pd.read_csv('demographics_' + id + '.csv')
  df_demographics = df_demographics.sort_values(['user_id', 'age'], ascending=[True, False])
  df_demographics = df_demographics.drop_duplicates('user_id',keep='first').reset_index(drop=True)
  new_vectors = pd.merge(vectors, df_demographics, on='user_id', how='left')
  new_vectors = drop_unnamed(new_vectors)
  return new_vectors

def pipeline1(business_id, alluser):
  ### READ THE RANKING FROM GTREE
  local_ranking_folder = gtree.loc[gtree['asin']==id, 'gfolder_rankings'].tolist()[0]
  local_ranking_files = drive.ListFile({'q': "'" + local_ranking_folder + "' in parents and trashed=false"}).GetList()

  download = drive.CreateFile({'id': local_ranking_files[0]['id']})
  download.GetContentFile(local_ranking_files[0]['title'])

  tmp_rel_ranking = pd.read_csv(local_ranking_files[0]['title'])
  tmp_rel_ranking = drop_unnamed(tmp_rel_ranking)
  
  tmp_date_ranking = tmp_rel_ranking.sort_values(by=['date'],ascending=True)
  tmp_rand_ranking = tmp_rel_ranking.sample(frac=1).reset_index(drop=True)

  df_rel_ranking = pd.DataFrame(columns=['user_id', 'user', 'comment', 'feedback', 'date','name','country'])
  counter = 0
  for i, rank in tmp_rel_ranking.iterrows():
    counter = counter + 1
    row = {
        'position':int(counter),
        'user_id':rank['user_id'],
        'user':rank['user'],
        'comment':rank['comment'],
        'feedback':rank['feedback'],
        'date':rank['date'],
        'name':rank['name'],
        'country':rank['country']
    }
    df_rel_ranking = df_rel_ranking.append(row, ignore_index=True)

  df_date_ranking = pd.DataFrame(columns=['user_id', 'user', 'comment', 'feedback', 'date','name','country'])
  counter = 0
  for i, rank in tmp_date_ranking.iterrows():
    counter = counter + 1
    row = {
        'position':int(counter),
        'user_id':rank['user_id'],
        'user':rank['user'],
        'comment':rank['comment'],
        'feedback':rank['feedback'],
        'date':rank['date'],
        'name':rank['name'],
        'country':rank['country']
    }
    df_date_ranking = df_date_ranking.append(row, ignore_index=True)

  df_rand_ranking = pd.DataFrame(columns=['user_id', 'user', 'comment', 'feedback', 'date','name','country'])
  counter = 0
  for i, rank in tmp_rand_ranking.iterrows():
    counter = counter + 1
    row = {
        'position':int(counter),
        'user_id':rank['user_id'],
        'user':rank['user'],
        'comment':rank['comment'],
        'feedback':rank['feedback'],
        'date':rank['date'],
        'name':rank['name'],
        'country':rank['country']
    }
    df_rand_ranking = df_rand_ranking.append(row, ignore_index=True)
  
  print("Ranking by Amazon filter:")
  pd.set_option('display.max_columns', None)
  print(df_rel_ranking)
  print('\n')
  print("Ranking by Date:")
  print(df_date_ranking)
  print('\n')
  print("Ranking Random:")
  print(df_rand_ranking)
  print('\n')

  ###

  df_rel_ranking = reorder_user_data(df_rel_ranking)
  df_date_ranking = reorder_user_data(df_date_ranking)
  df_rand_ranking = reorder_user_data(df_rand_ranking)
    

  print("\n++++++++++++++++ RANKING ++++++++++++++++++\n")

  print("Ranking by Amazon filter:")
  pd.set_option('display.max_columns', None)
  print(df_rel_ranking)
  print('\n')
  print("Ranking by Date:")
  print(df_date_ranking)
  print('\n')
  print("Ranking Random:")
  print(df_rand_ranking)
  print('\n')

  return df_rel_ranking, df_date_ranking, df_rand_ranking


def pipeline2(df_ranking_by_relevance, id, N_of_groups,
              local_list_of_attributes, method, alluser, destination, assumption):
  print("\n++++++++++++++++ GROUPS CREATION ++++++++++++++++++\n")
  print(id)
  if alluser:
    df_vectors = create_vectors_yelp(df_ranking_by_relevance, id, local_list_of_attributes,
                                     method, destination)
    
  if method == 'custom':
    df_groups, destination = create_groups_custom(N_of_groups,local_list_of_attributes,id, destination, assumption)
  
  percents = compute_groups_percents(df_groups, N_of_groups)
  
  print(df_groups)
  print(percents)
  return df_groups, percents, destination


def pipeline3(business_id, method, df_ranking_by_relevance, df_ranking_by_date, 
              df_ranking_by_random, percents, list_of_attributes,
              alluser, destination, df_groups):
  
  df_result = pd.DataFrame(columns=['Exposure_method', 'Context', 'Means', 'P-value'])
  df_result.to_csv('result.csv')
  id_new_file = upload_file('result.csv', destination)


  print("\n++++++++++++++++ EXPOSURE CALCULATION ++++++++++++++++++\n")
  
  print("----------------DEMOGRAPHIC PARITY EXP------------------\n")
  print("------------Ranking by Amazon filter:------------\n")
  yelp_exposures, yelp_user_exposures = print_demographic_parity_exposure(business_id,
                                                    method, df_ranking_by_relevance,
                                                     "yelp_", list_of_attributes, destination)
  print("------------Ranking by Date:------------\n")
  date_exposures, date_user_exposures = print_demographic_parity_exposure(business_id,
                                                    method, df_ranking_by_date,
                                                     "date_", list_of_attributes, destination)
  print("------------Ranking Random:------------\n")
  random_exposures, rand_user_exposures = print_demographic_parity_exposure(business_id,
                                                      method, df_ranking_by_random,
                                                       "rand_", list_of_attributes, destination)
  
  array_group_id = np.arange(0,df_groups['group_id'].max()+1)
  filePath = stat_significance_inter_rankings(yelp_user_exposures, rand_user_exposures, method,
                                   "demgr_yelp_", business_id, array_group_id, destination) # group_id
  upload_file(filePath, destination)
  filePath = stat_significance_inter_rankings(date_user_exposures, rand_user_exposures, method,
                                   "demgr_date_", business_id, array_group_id, destination)
  upload_file(filePath, destination)

  if alluser and not set(list_of_attributes).intersection(set(['age', 'gender', 'ethnicity'])):
    yelp_error = st.pstdev(yelp_user_exposures['exposure'])
    date_error = st.pstdev(date_user_exposures['exposure'])
    rand_error = st.pstdev(rand_user_exposures['exposure'])
  else:
    yelp_error = st.stdev(yelp_user_exposures['exposure'])
    date_error = st.stdev(date_user_exposures['exposure'])
    rand_error = st.stdev(rand_user_exposures['exposure'])
  
  get_plots(yelp_exposures, date_exposures, random_exposures, percents,
            'plot_demgr_' + method + '_' + business_id, list_of_attributes,
            method, business_id, alluser, [yelp_error, date_error, rand_error], destination)
  get_scatter_plots(yelp_user_exposures, date_user_exposures, random_exposures,
                    'scatter_demgr_' + method + '_' + business_id, list_of_attributes,
                    method, business_id, alluser, destination)
  get_distribution_plots(yelp_user_exposures, date_user_exposures, random_exposures,
                    'distribution_demgr_' + method + '_' + business_id, list_of_attributes,
                    method, business_id, alluser, destination)
  

  print("----------------DISPARATE TREATMENT EXP------------------\n")
  print("------------Ranking by Amazon filter:------------\n")
  yelp_exposures, yelp_user_exposures = print_disparate_treatment_exposure(business_id,
                                                    method, df_ranking_by_relevance,
                                                     "yelp_", list_of_attributes, alluser, destination)
  print("------------Ranking by Date:------------\n")
  date_exposures, date_user_exposures = print_disparate_treatment_exposure(business_id,
                                                    method, df_ranking_by_date,
                                                     "date_", list_of_attributes, alluser, destination)
  print("------------Ranking Random:------------\n")
  random_exposures, rand_user_exposures = print_disparate_treatment_exposure(business_id,
                                                      method, df_ranking_by_random,
                                                       "rand_", list_of_attributes, alluser, destination)
  
  array_group_id = np.arange(0,df_groups['group_id'].max()+1)
  filePath = stat_significance_inter_rankings(yelp_user_exposures, rand_user_exposures, method,
                                   "disptr_yelp_", business_id, array_group_id, destination) # group_id
  upload_file(filePath, destination)
  filePath = stat_significance_inter_rankings(date_user_exposures, rand_user_exposures, method,
                                   "disptr_date_", business_id, array_group_id, destination)
  upload_file(filePath, destination)

  if alluser and not set(list_of_attributes).intersection(set(['age', 'gender', 'ethnicity'])):
    yelp_error = st.pstdev(yelp_user_exposures['exposure'])
    date_error = st.pstdev(date_user_exposures['exposure'])
    rand_error = st.pstdev(rand_user_exposures['exposure'])
  else:
    yelp_error = st.stdev(yelp_user_exposures['exposure'])
    date_error = st.stdev(date_user_exposures['exposure'])
    rand_error = st.stdev(rand_user_exposures['exposure'])
  
  get_plots(yelp_exposures, date_exposures, random_exposures, percents,
            'plot_disptr_' + method + '_' + business_id, list_of_attributes,
            method, business_id, alluser, [yelp_error, date_error, rand_error], destination)
  get_scatter_plots(yelp_user_exposures, date_user_exposures, rand_user_exposures,
                    'scatter_disptr_' + method + '_' + business_id, list_of_attributes,
                    method, business_id, alluser, destination)
  get_distribution_plots(yelp_user_exposures, date_user_exposures, random_exposures,
                    'distribution_disptr_' + method + '_' + business_id, list_of_attributes,
                    method, business_id, alluser, destination)




  print("----------------DISPARATE IMPACT EXP------------------\n")
  print("------------Ranking by Yelp filter:------------\n")
  yelp_exposures, yelp_user_exposures = print_disparate_impact_exposure(business_id, method, df_ranking_by_relevance,
                                                   "yelp_", list_of_attributes, alluser, destination)
  
  print("------------Ranking by Date:------------\n")
  date_exposures, date_user_exposures = print_disparate_impact_exposure(business_id, method, df_ranking_by_date,
                                                   "date_", list_of_attributes, alluser, destination)
  
  print("------------Ranking Random:------------\n")
  random_exposures, rand_user_exposures = print_disparate_impact_exposure(business_id, method, df_ranking_by_random,
                                                     "rand_", list_of_attributes, alluser, destination)
  
  filePath = stat_significance_inter_rankings(yelp_user_exposures, rand_user_exposures, method,
                                   "dispimp_yelp_", business_id, array_group_id, destination)
  upload_file(filePath, destination)
  filePath = stat_significance_inter_rankings(date_user_exposures, rand_user_exposures, method,
                                   "dispimp_date_", business_id, array_group_id, destination)
  upload_file(filePath, destination)

  if alluser and not set(list_of_attributes).intersection(set(['age', 'gender', 'ethnicity'])):
    yelp_error = st.pstdev(yelp_user_exposures['exposure'])
    date_error = st.pstdev(date_user_exposures['exposure'])
    rand_error = st.pstdev(rand_user_exposures['exposure'])
  else:
    yelp_error = st.stdev(yelp_user_exposures['exposure'])
    date_error = st.stdev(date_user_exposures['exposure'])
    rand_error = st.stdev(rand_user_exposures['exposure'])
  
  get_plots(yelp_exposures, date_exposures, random_exposures, percents,
            'plot_dispimp_' + method + '_' + business_id, list_of_attributes,
            method, business_id, alluser, [yelp_error, date_error, rand_error], destination)
  get_scatter_plots(yelp_user_exposures, date_user_exposures, rand_user_exposures,
                    'scatter_dispimp_' + method + '_' + business_id, list_of_attributes,
                    method, business_id, alluser, destination)
  get_distribution_plots(yelp_user_exposures, date_user_exposures, random_exposures,
                    'distribution_dispimp_' + method + '_' + business_id, list_of_attributes,
                    method, business_id, alluser, destination)


def create_groups_custom(N_of_groups, list_of_attributes, id, destination, assumption):
  df_vectors = pd.read_csv("user_vectors_" + id + ".csv")

  local_list_of_attributes = list_of_attributes

  if set(list_of_attributes).intersection(set(['age', 'gender', 'ethnicity', 'review_sentiment'])):
    text_attribute_list = ['gender', 'ethnicity']
    df_vectors, dummy_columns_name = generate_dummies(df_vectors, text_attribute_list)
    # list_of_attributes - text_attribute_list + dummy_columns_name
    # subtraction
    temp = [item for item in list_of_attributes if item not in text_attribute_list]
    list_of_attributes = temp + dummy_columns_name
    #serve per la descrizione dei gruppi, da fare

    #EXCLUDE USERS WITH NO INFO, CLUSTER THE REMAINING IN C-1
    df_no_info = df_vectors[pd.isna(df_vectors['gender'])]
    df_yes_info = df_vectors[pd.notna(df_vectors['gender'])]
    print('All users =', len(df_vectors))
    print('Users with info =',len(df_yes_info.index))
    print('User without info =',len(df_no_info.index))
  
  else:
    df_no_info = pd.DataFrame()
    df_yes_info = df_vectors

  
  # GRUPPI PER GENERE
  if local_list_of_attributes == ['gender']:
    if assumption=='':
      femmine = df_yes_info[df_yes_info['gender']=='female']
      maschi = df_yes_info[df_yes_info['gender']=='male']
      new_df_users = df_yes_info[['user_id']].reset_index(drop=True)
      new_df_users['group_id'] = np.NaN
      for j, user in femmine.iterrows():
        new_df_users.loc[new_df_users['user_id'] == user['user_id'], 'group_id'] = 1
        
      for j, user in maschi.iterrows():
        new_df_users.loc[new_df_users['user_id'] == user['user_id'], 'group_id'] = 2
    else:
      new_df_users, destination = do_assumptions(df_yes_info, df_no_info,
                                                 destination, assumption)
      df_no_info = df_no_info.iloc[0:0]

  # GRUPPI PER REVIEW SENTIMENT
  if local_list_of_attributes == ['review_sentiment']:
    df_yes_info = df_yes_info[['user_id', 'review_sentiment']]
    print(df_yes_info)
    positivi = df_yes_info[df_yes_info['review_sentiment'].str.startswith('positive')]
    negativi = df_yes_info[df_yes_info['review_sentiment'].str.startswith('negative')]
    neutrali = df_yes_info[df_yes_info['review_sentiment'].str.startswith('neutral')]
    new_df_users = df_yes_info[['user_id']].reset_index(drop=True)
    new_df_users['group_id'] = np.NaN
    for j, user in positivi.iterrows():
      new_df_users.loc[new_df_users['user_id'] == user['user_id'], 'group_id'] = 1
      
    for j, user in negativi.iterrows():
      new_df_users.loc[new_df_users['user_id'] == user['user_id'], 'group_id'] = 2
    
    for j, user in neutrali.iterrows():
      new_df_users.loc[new_df_users['user_id'] == user['user_id'], 'group_id'] = 3
    
  # GRUPPI PER ETNIA
  if local_list_of_attributes == ['ethnicity']:
    bianchi = df_yes_info[df_yes_info['ethnicity']=='white']
    neri = df_yes_info[df_yes_info['ethnicity']=='black or african american']
    altri = df_yes_info[(df_yes_info['ethnicity']=='hispanic, latino, or spanish origin') | (df_yes_info['ethnicity']=='asian') | (df_yes_info['ethnicity']=='middle eastern or north african') | (df_yes_info['ethnicity']=='native hawaiian or pacific islander')]
    new_df_users = df_yes_info[['user_id']].reset_index(drop=True)
    new_df_users['group_id'] = np.NaN
    for j, user in bianchi.iterrows():
      new_df_users.loc[new_df_users['user_id'] == user['user_id'], 'group_id'] = 1
      
    for j, user in neri.iterrows():
      new_df_users.loc[new_df_users['user_id'] == user['user_id'], 'group_id'] = 2
    
    for j, user in altri.iterrows():
      new_df_users.loc[new_df_users['user_id'] == user['user_id'], 'group_id'] = 3

  # GRUPPI PER ETNIA, GENERE
  if local_list_of_attributes == ['gender','ethnicity']:
    bianchi = df_yes_info[df_yes_info['ethnicity']=='white']
    b_under_39 = bianchi[(bianchi['age']<39)]
    b_over_39 = bianchi[(bianchi['age']>=39)]
    b_fem = bianchi[bianchi['gender']=='female'].reset_index(drop=True)
    b_mas = bianchi[bianchi['gender']=='male'].reset_index(drop=True)
    
    neri = df_yes_info[df_yes_info['ethnicity']=='black or african american']
    n_under_39 = neri[(neri['age']<39)]
    n_over_39 = neri[(neri['age']>=39)]
    n_fem = neri[neri['gender']=='female'].reset_index(drop=True)
    n_mas = neri[neri['gender']=='male'].reset_index(drop=True)
    
    altri = df_yes_info[(df_yes_info['ethnicity']=='hispanic, latino, or spanish origin') | (df_yes_info['ethnicity']=='asian') | (df_yes_info['ethnicity']=='middle eastern or north african') | (df_yes_info['ethnicity']=='native hawaiian or pacific islander')]
    a_under_39 = altri[(altri['age']<39)]
    a_over_39 = altri[(altri['age']>=39)]
    a_fem = altri[altri['gender']=='female'].reset_index(drop=True)
    a_mas = altri[altri['gender']=='male'].reset_index(drop=True)
    new_df_users = df_yes_info[['user_id']].reset_index(drop=True)
    new_df_users['group_id'] = np.NaN
    for j, user in b_fem.iterrows():
      new_df_users.loc[new_df_users['user_id'] == user['user_id'], 'group_id'] = 1
      
    for j, user in b_mas.iterrows():
      new_df_users.loc[new_df_users['user_id'] == user['user_id'], 'group_id'] = 2
    
    for j, user in n_fem.iterrows():
      new_df_users.loc[new_df_users['user_id'] == user['user_id'], 'group_id'] = 3
      
    for j, user in n_mas.iterrows():
      new_df_users.loc[new_df_users['user_id'] == user['user_id'], 'group_id'] = 4
      
    for j, user in a_fem.iterrows():
      new_df_users.loc[new_df_users['user_id'] == user['user_id'], 'group_id'] = 5
      
    for j, user in a_mas.iterrows():
      new_df_users.loc[new_df_users['user_id'] == user['user_id'], 'group_id'] = 6

  if not df_no_info.empty:
    # ADD THE EXCLUDED IN LAST C
    new_df_users = pd.merge(new_df_users,df_no_info,how='outer')
    new_df_users = drop_unnamed(new_df_users)
    new_df_users = new_df_users.fillna(0)

  #destination = set_file_destination(local_list_of_attributes, 'custom', id)

  new_df_users.to_csv('groups_custom_' + id + '.csv')
  upload_file('groups_custom_' + id + '.csv',destination)
  return new_df_users, destination


def do_assumptions(df_yes, df_no_info, destination, assumption):
  total = len(df_yes.index)+len(df_no_info.index)
  already_men = len(df_yes[df_yes['gender']=='male'].values.tolist())
  # ALL IN ONE GROUP
  if assumption=='all_men':
    id_new_folder = create_folder_in_drive(drive, 'All_men', destination)
    df_no = df_no_info
    df_no['gender'].fillna('male', inplace = True)
    df = pd.concat([df_yes, df_no])
  if assumption=='all_women':
    id_new_folder = create_folder_in_drive(drive, 'All_women', destination)
    df_no = df_no_info
    df_no['gender'] = 'female'
    df = pd.concat([df_yes, df_no])
  # 50%
  if assumption=='50':
    men_size = total//2
    id_new_folder = create_folder_in_drive(drive, '50and50', destination)
    df_no = df_no_info
    while (already_men+len(df_no[df_no['gender']=='male'].values.tolist()))<=men_size:
      index_found = df_no.index[df_no['gender'].isnull()].tolist()[0]
      df_no.loc[index_found, 'gender'] = 'male'
    df_no['gender'].fillna('male', inplace = True)
    df = pd.concat([df_yes, df_no])
  # EQUALLY DISTRIBUTED
  if assumption=='equal':
    id_new_folder = create_folder_in_drive(drive, 'Equally_distributed', destination)
    df_no = df_no_info
    even = 0
    for ind, row in df_no.iterrows():
      if even==0:
        even=1
        df_no.loc[ind,'gender']='male'
      else:
        df_no.loc[ind,'gender']='female'
        even=0
    df = pd.concat([df_yes, df_no])
  # MAINTAIN PROPORTION
  if assumption=='proportioned':
    #male_size:partial=new_male_size:total
    partial = len(df_yes.index)
    men_size = (total*(len(df_yes[df_yes['gender']=='male'].reset_index(drop=True).index)))//partial
    id_new_folder = create_folder_in_drive(drive, 'Maintaining_proportion', destination)
    df_no = df_no_info
    while (already_men+len(df_no[df_no['gender']=='male'].values.tolist()))<=men_size:
      index_found = df_no.index[df_no['gender'].isnull()].tolist()[0]
      df_no.loc[index_found, 'gender'] = 'male'
    df_no['gender'].fillna('female', inplace = True)
    df = pd.concat([df_yes, df_no])
  femmine = df[df['gender']=='female']
  maschi = df[df['gender']=='male']

  new_df_users = df[['user_id']].reset_index(drop=True)
  new_df_users['group_id'] = np.NaN
  for j, user in femmine.iterrows():
    new_df_users.loc[new_df_users['user_id'] == user['user_id'], 'group_id'] = 0
  for j, user in maschi.iterrows():
    new_df_users.loc[new_df_users['user_id'] == user['user_id'], 'group_id'] = 1
  return new_df_users, id_new_folder


def print_disparate_impact_exposure(business_id, method,  df_ranking,
                                    filename, list_of_attributes, alluser, destination):
  df_groups = pd.read_csv('groups_' + method + '_' + business_id + '.csv')
  i = 0
  exposures = pd.DataFrame(columns=['group_id', 'exposure'])
  user_exposures = pd.DataFrame(columns=['user_id', 'group_id', 'exposure'])
  user_exposures['user_id'] = df_ranking['user_id']
  while i <= df_groups['group_id'].max():
      current_exp, user_exposures = disparate_impact_exposure(df_groups[df_groups['group_id'] == i], df_ranking,user_exposures, business_id, i, alluser)
      exposures.loc[i, 'group_id'] = i
      exposures.loc[i, 'exposure'] = current_exp
      i = i + 1
  
  #destination = set_file_destination(list_of_attributes, method, business_id)
  user_exposures.to_csv('user_exp_dispimp_' + method + '_' + filename + business_id + '.csv')
  upload_file('user_exp_dispimp_' + method + '_' + filename + business_id + '.csv',
                  destination)
  
  filePath = stat_significance_inter_groups(df_groups, 'dispimp_'+filename, business_id,
                                            method, user_exposures, destination)

  exposures.to_csv('exp_dispimp_' + method + '_' + filename + business_id + '.csv')
  upload_file('exp_dispimp_' + method + '_' + filename + business_id + '.csv',
                  destination)
  upload_file(filePath, destination)
  print(exposures)
  print('\n')
  return exposures, user_exposures


def print_demographic_parity_exposure(business_id, method, df_ranking, filename,
                                      list_of_attributes, destination):
  df_groups = pd.read_csv('groups_' + method + '_' + business_id + '.csv')
  i = 0
  exposures = pd.DataFrame(columns=['group_id', 'exposure'])
  user_exposures = pd.DataFrame(columns=['user_id', 'group_id', 'exposure'])
  print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
  print(df_ranking)
  user_exposures['user_id'] = df_ranking['user_id']
  while i <= df_groups['group_id'].max():
      current_exp, user_exposures = demographic_parity_exposure(df_groups[df_groups['group_id'] == i], df_ranking, user_exposures, i)
      exposures.loc[i, 'group_id'] = i
      exposures.loc[i, 'exposure'] = current_exp
      i = i + 1
  
  #destination = set_file_destination(list_of_attributes, method, business_id)
  user_exposures.to_csv('user_exp_demgr_' + method + '_' + filename + business_id + '.csv')
  upload_file('user_exp_demgr_' + method + '_' + filename + business_id + '.csv',
                  destination)
  filePath = stat_significance_inter_groups(df_groups, 'demgr_'+filename, business_id,
                                            method, user_exposures, destination)

  exposures.to_csv('exp_demgr_' + method + '_' + filename + business_id + '.csv')
  upload_file('exp_demgr_' + method + '_' + filename + business_id + '.csv',
                  destination)
  upload_file(filePath, destination)
  print(exposures)
  print('\n')
  return exposures, user_exposures
  
def print_disparate_treatment_exposure(business_id, method, df_ranking, filename,
                                      list_of_attributes, alluser, destination):
  df_groups = pd.read_csv('groups_' + method + '_' + business_id + '.csv')
  i = 0
  exposures = pd.DataFrame(columns=['group_id', 'exposure'])
  user_exposures = pd.DataFrame(columns=['user_id', 'group_id', 'exposure'])
  user_exposures['user_id'] = df_ranking['user_id']
  while i <= df_groups['group_id'].max():
      current_exp, user_exposures = disparate_treatment_exposure(df_groups[df_groups['group_id'] == i], df_ranking,user_exposures, business_id, i, alluser)
      exposures.loc[i, 'group_id'] = i
      exposures.loc[i, 'exposure'] = current_exp
      i = i + 1
  
  #destination = set_file_destination(list_of_attributes, method, business_id)
  user_exposures.to_csv('user_exp_disptr_' + method + '_' + filename + business_id + '.csv')
  upload_file('user_exp_disptr_' + method + '_' + filename + business_id + '.csv',
                  destination)
  filePath = stat_significance_inter_groups(df_groups, 'disptr_'+filename, business_id,
                                            method, user_exposures, destination)
  exposures.to_csv('exp_disptr_' + method + '_' + filename + business_id + '.csv')
  upload_file('exp_disptr_' + method + '_' + filename + business_id + '.csv',
                  destination)
  upload_file(filePath, destination)
  print(exposures)
  print('\n')
  return exposures, user_exposures

def stat_significance_inter_groups(df_groups, filename, business_id, method, df_user_exposures, destination):
  array_group_id = np.arange(0,df_groups['group_id'].max()+1)
  couples = list(itertools.combinations(array_group_id,2))
  couples = [(x,y) for (x,y) in couples if x!=y]
  print("Group couples:", couples) # [(0,1), (0,2), (1,2)]
  filePath = 'stat_groups_' + method + '_' + filename + business_id + '.txt'
  if os.path.exists(filePath):
    os.remove(filePath)
  ciclo = 0
  pvalues = []
  while ciclo < 3:
    with open(filePath, 'a') as output:
        output.write('Without first and last ' + str(ciclo) + ' rows:\n\n')
    user_exposures = df_user_exposures.iloc[ciclo:(len(df_user_exposures.index)-ciclo)]
    print('Size:', len(user_exposures.index))
    for id1,id2 in couples:
      group1 = user_exposures[user_exposures['group_id'] == id1]['exposure']
      group2 = user_exposures[user_exposures['group_id'] == id2]['exposure']
      array1 = group1.values
      array2 = group2.values
      result = stats.ks_2samp(array1, array2)
      pvalues.append(result[1])
      percent1 = 100*(len(group1.index)/len(user_exposures.index))
      percent2 = 100*(len(group2.index)/len(user_exposures.index))
      with open(filePath, 'a') as output:
        output.write('Percent GROUP '+ str(id1) +': ' + str(percent1) + '\nPercent GROUP '+
                    str(id2) +': ' + str(percent2) + '\nResult: ')
        output.write(str(result)+'\n\n')
    with open(filePath, 'a') as output:
        output.write('----------------------------------------\n\n')
    ciclo = ciclo + 1
  update_result_table_inter_groups(filename[:-6], filename[-5:-1], couples, pvalues, destination)
  return filePath
  
def update_result_table_inter_groups(exp_method, ranking, couple_groups, p_values, destination):
  file_list = drive.ListFile({'q': "'" + destination + "' in parents and trashed=false"}).GetList()
  id_result_file = ''
  for file in file_list:
    if file['title']=='pvalue_inter_groups.csv':
      id_result_file = file['id']
  
  if id_result_file == '':
    df = pd.DataFrame(columns=['Exposure method', 'Ranking', 'Groups', 'P-value 1', 'P-value 2', 'P-value 3'])
  else:
    download = drive.CreateFile({'id': id_result_file})
    download.GetContentFile('pvalue_inter_groups.csv')
    df = pd.read_csv('pvalue_inter_groups.csv')

  if ranking == "yelp":
    ranking = "Yelp"
  if ranking == "date":
    ranking = "Date"
  if ranking == "rand":
    ranking = "Random"
  if exp_method == "demgr":
    exp_method = "Demographic Parity"
  if exp_method == "disptr":
    exp_method = "Disparate Treatment"
  if exp_method == "dispimp":
    exp_method = "Disparate Impact"
  i = 0
  for couple in couple_groups:
    new_row = {'Exposure method':exp_method, 'Ranking':ranking, 'Groups':str(couple[0])+' - '+str(couple[1]), 
               'P-value 1':p_values[i], 'P-value 2':p_values[i+len(couple_groups)], 'P-value 3':p_values[i+(len(couple_groups)*2)]}
    i = i + 1
    df = df.append(new_row, ignore_index=True)
    df = drop_unnamed(df)
  
  df.to_csv('pvalue_inter_groups.csv')
  upload_file('pvalue_inter_groups.csv', destination)

def stat_significance_inter_rankings(df_user_exposures, df_random_user_exposures, method,
                                     filename, business_id, group_ids, destination):
  filePath = 'stat_rankings_' + method + '_' + filename + business_id + '.txt'
  if os.path.exists(filePath):
    os.remove(filePath)
  ciclo = 0
  pvalues = []
  while ciclo < 3:
    with open(filePath, 'a') as output:
        output.write('Without first and last ' + str(ciclo) + ' rows:\n\n')
    user_exposures = df_user_exposures.iloc[ciclo:(len(df_user_exposures.index)-ciclo)]
    random_user_exposures = df_random_user_exposures.iloc[ciclo:(len(df_random_user_exposures.index)-ciclo)]
    for group_id in group_ids:
      group1 = user_exposures[user_exposures['group_id'] == group_id]['exposure']
      group2 = random_user_exposures[random_user_exposures['group_id'] == group_id]['exposure']
      array1 = group1.values
      array2 = group2.values
      result = stats.ks_2samp(array1, array2)
      pvalues.append(result[1])
      percent1 = 100*(len(group1.index)/len(user_exposures.index))
      percent2 = 100*(len(group2.index)/len(random_user_exposures.index))
      print('+++++++++TEST STAT SIGNIFICANCE INTER RANKINGS+++++++++')
      print('RESULT:')
      print(result)
      with open(filePath, 'a') as output:
        output.write('Percent GROUP '+ str(group_id) +': ' + str(percent1) + '\nResult: ')
        output.write(str(result)+'\n\n')
    with open(filePath, 'a') as output:
        output.write('----------------------------------------\n\n')
    ciclo = ciclo + 1
  update_result_table_inter_rankings(filename[:-6], group_ids, (filename[-5:-1],'Random'), pvalues, destination)
  return filePath

def update_result_table_inter_rankings(exp_method, group_ids, rankings, p_values, destination):
  file_list = drive.ListFile({'q': "'" + destination + "' in parents and trashed=false"}).GetList()
  id_result_file = ''
  for file in file_list:
    if file['title']=='pvalue_inter_rankings.csv':
      id_result_file = file['id']
  
  if id_result_file == '':
    df = pd.DataFrame(columns=['Exposure method', 'Rankings', 'Group', 'P-value 1', 'P-value 2', 'P-value 3'])
  else:
    download = drive.CreateFile({'id': id_result_file})
    download.GetContentFile('pvalue_inter_rankings.csv')
    df = pd.read_csv('pvalue_inter_rankings.csv')

  if rankings[0] == "yelp":
    ranking1 = "Yelp"
  if rankings[0] == "date":
    ranking1 = "Date"

  ranking2 = "Random"

  if exp_method == "demgr":
    exp_method = "Demographic Parity"
  if exp_method == "disptr":
    exp_method = "Disparate Treatment"
  if exp_method == "dispimp":
    exp_method = "Disparate Impact"
  i = 0
  for group_id in group_ids:
    new_row = {'Exposure method':exp_method, 'Rankings':ranking1+' - '+ranking2, 'Group': group_id,
               'P-value 1':p_values[i], 'P-value 2':p_values[i+len(group_ids)], 'P-value 3':p_values[i+(len(group_ids)*2)]}
    i = i + 1
    df = df.append(new_row, ignore_index=True)
    df = drop_unnamed(df)
  
  df.to_csv('pvalue_inter_rankings.csv')
  upload_file('pvalue_inter_rankings.csv', destination)
  
def disparate_treatment_exposure(df_group, ranking, user_exposures, business_id, group_index, alluser):
  all_exposures = []
  for i, user in df_group.iterrows():
    user_id = user['user_id']

    if alluser:
      useful = ranking[ranking['user_id'] == user_id]['useful_votes'].values[0]
    
    counts = useful + 2
    
    base = 2  # con 10 i valori sono troppo bassi
    counts = math.log(counts, base)
    
    position = ranking[ranking['user_id'] == user_id]['position'].values[0]
    
    current_exp = exp(position) / counts
    
    all_exposures.append(current_exp)
    user_exposures.loc[user_exposures['user_id'] == user_id, 'exposure'] = current_exp
    user_exposures.loc[user_exposures['user_id'] == user_id, 'group_id'] = group_index
  print('all_exposures size:', len(all_exposures))
  mean = st.mean(all_exposures)
  return mean, user_exposures

def update_result_table(exp_method, context, means, p_value, destination):
  file_list = drive.ListFile({'q': "'" + destination + "' in parents and trashed=false"}).GetList()
  for file in file_list:
    if file['title']=='result.csv':
      id_result_file = file['id']
  download = drive.CreateFile({'id': id_result_file}) # id file gtree.csv
  download.GetContentFile('result.csv')
  df = pd.read_csv('result.csv')
  new_row = {'Exposure_method':exp_method, 'Context':context,
             'Means':str(means[0])+' - '+str(means[1]), 'P-value':p_value}
  df = df.append(new_row, ignore_index=True)
  df = drop_unnamed(df)
  df = drop_unnamed(df)
  df.to_csv('result.csv')
  upload_file('result.csv', destination) # id folder data


def disparate_impact_exposure(df_group, ranking, user_exposures, business_id, group_index, alluser):
  primo = ranking['useful_votes'].values.tolist()
  norm = 1
  for x in primo:
    temp = math.log(x+2, 2)
    if temp > norm:
      norm = temp
  
  all_exposures = []
  for i, user in df_group.iterrows():
    user_id = user['user_id']

    if alluser:
      useful = ranking[ranking['user_id'] == user_id]['useful_votes'].values[0]

    
    counts = useful + 2
    
    base = 2  # con 10 i valori sono troppo bassi
    counts = math.log(counts, base)
    #counts = counts/norm
    
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print(df_group)
    print(ranking)
    position = ranking[ranking['user_id'] == user_id]['position'].values[0]
    
    current_exp = exp(position) * counts
    
    all_exposures.append(current_exp)
    user_exposures.loc[user_exposures['user_id'] == user_id, 'exposure'] = current_exp
    user_exposures.loc[user_exposures['user_id'] == user_id, 'group_id'] = group_index
  print('all_exposures size:', len(all_exposures))
  mean = st.mean(all_exposures)
  return mean, user_exposures


def demographic_parity_exposure(df_group, ranking, user_exposures, group_index):
    all_exposures = []
    for i, user in df_group.iterrows():
        user_id = user['user_id']
        position = ranking[ranking['user_id'] == user_id]['position'].values[0]
        current_exp = exp(position)
        all_exposures.append(current_exp)
        user_exposures.loc[user_exposures['user_id'] == user_id, 'exposure'] = current_exp
        user_exposures.loc[user_exposures['user_id'] == user_id, 'group_id'] = group_index
    print('all_exposures size:', len(all_exposures))
    mean = st.mean(all_exposures)
    return mean, user_exposures


def exp(position):
    if position == 'no match':
        return 0
    else:
        return 1/(math.log(1 + position, 2))

		
def get_plots(yelp_exposures, date_exposures, random_exposures, percents, title,
              list_of_attributes, method, business_id, alluser, errors, destination):
  width = 0.20
  y_min = 0.0
  y_max = 0.5 #1.2 

  x1 = [el['group_id'] - width for i, el in yelp_exposures[['group_id']].iterrows()]
  x2 = [el['group_id'] for i, el in date_exposures[['group_id']].iterrows()]
  x3 = [el['group_id'] + width for i, el in random_exposures[['group_id']].iterrows()]

  y1 = [el['exposure'] for i, el in yelp_exposures[['exposure']].iterrows()]
  y2 = [el['exposure'] for i, el in date_exposures[['exposure']].iterrows()]
  y3 = [el['exposure'] for i, el in random_exposures[['exposure']].iterrows()]

  print(y1)

  '''y1_error = np.std(y1)
  y2_error = np.std(y2)
  y3_error = np.std(y3)'''
  '''# I have missing user when using dataset OR using demographic attributes
  if alluser and set(list_of_attributes).intersection(set(['age', 'gender', 'ethnicity'])):
    y1_error = st.pstdev(y1)
    y2_error = st.pstdev(y2)
    y3_error = st.pstdev(y3)
  else:
    y1_error = st.stdev(y1)
    y2_error = st.stdev(y2)
    y3_error = st.stdev(y3)'''

  plt.bar(x1,y1,width=width,align='center', color='red', label='amazon', yerr=errors[0], capsize=5)
  plt.bar(x2,y2,width=width,align='center', color='green', label='date', yerr=errors[1], capsize=5)
  plt.bar(x3,y3,width=width,align='center', color='blue', label='random', yerr=errors[2], capsize=5)
  plt.legend(loc="upper center")
  plt.xlabel('Group id')
  if title.startswith("plot_demgr_"):
    plt.ylabel('Exposure (Demographic Parity)')
  elif title.startswith("plot_dispimp_"):
    #plt.ylabel('Exposure (Disparate Impact)')
    plt.ylabel('Exposure (Disparate Impact Normalized)')
  elif title.startswith("plot_disptr_"):
    plt.ylabel('Exposure (Disparate Treatment)')
  else: plt.ylabel('Exposure')

  if len(list_of_attributes)==2:
    id_labels=["u","f/w","m/w","f/b","m/b","f/o","m/o"]
  if len(list_of_attributes)==1:
    id_labels=["unknown","woman","man"]
  this_range = [str(id_labels[int(id)]) + ":" + "{:.{}f}".format(percent,1) + "%" for id, percent in zip(np.arange(min(x2), max(x2)+1, 1.0), percents)]
  print("this_range")
  print(this_range)
  plt.xticks(np.arange(min(x2), max(x2)+1, 1),this_range)
  plt.yticks(np.arange(y_min, y_max, 0.05))#0.1
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()

  #destination = set_file_destination(list_of_attributes, method, business_id)
  
  #plt.show()
  plt.savefig(title + '.png')
  upload_file(title + '.png', destination)
  plt.close()


def get_scatter_plots(yelp_user_exposures, date_user_exposures, random_user_exposures, title,
              list_of_attributes, method, business_id, alluser, destination):
  width = 0.20
  y_min = 0.0
  y_max = 1.5

  x1 = [el['group_id'] - width for i, el in yelp_user_exposures[['group_id']].iterrows()]
  x2 = [el['group_id'] for i, el in date_user_exposures[['group_id']].iterrows()]
  x3 = [el['group_id'] + width for i, el in random_user_exposures[['group_id']].iterrows()]

  y1 = [el['exposure'] for i, el in yelp_user_exposures[['exposure']].iterrows()]
  y2 = [el['exposure'] for i, el in date_user_exposures[['exposure']].iterrows()]
  y3 = [el['exposure'] for i, el in random_user_exposures[['exposure']].iterrows()]

  plt.scatter(x1,y1, s=10, color='red', label='amazon')
  plt.scatter(x2,y2, s=10, color='green', label='date')
  plt.scatter(x3,y3, s=10, color='blue', label='random')
  plt.legend(loc="upper center")
  plt.xlabel('Group id')
  plt.ylabel('Exposure')
  #this_range = [str(int(id)) + ":" + "{:.{}f}".format(percent,1) + "%" for id, percent in zip(np.arange(min(x2), max(x2)+1, 1.0), percents)]
  plt.xticks(np.arange(min(x2), max(x2)+1, 1.0))
  plt.yticks(np.arange(y_min, y_max, 0.1))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()

  #destination = set_file_destination(list_of_attributes, method, business_id)
  
  #plt.show()
  plt.savefig(title + '.png')
  upload_file(title + '.png', destination)
  plt.close()

def get_distribution_plots(yelp_user_exposures, date_user_exposures, random_user_exposures, title,
              list_of_attributes, method, business_id, alluser, destination):
  print(yelp_user_exposures)
  asdrubale = 0
  if asdrubale == 1:
    width = 0.20
    y_min = 0.0
    y_max = 1.5

    y1 = [el['exposure'] for i, el in yelp_user_exposures[['exposure']].iterrows()]
    y2 = [el['exposure'] for i, el in date_user_exposures[['exposure']].iterrows()]
    y3 = [el['exposure'] for i, el in random_user_exposures[['exposure']].iterrows()]

    x1 = y1
    x1.reverse()
    
    x2 = y2
    x2.reverse()

    x3 = y3
    x3.reverse()

    print(x1)
    print(y1)

    plt.scatter(x1,y1, s=10, color='red', label='amazon')
    plt.scatter(x2,y2, s=10, color='green', label='date')
    plt.scatter(x3,y3, s=10, color='blue', label='random')
    plt.legend(loc="upper center")
    #plt.xlabel('boh')
    if title.startswith("plot_demgr_"):
      plt.ylabel('Exposure (Demographic Parity)')
    elif title.startswith("plot_dispimp_"):
      plt.ylabel('Exposure (Disparate Impact)')
      #plt.ylabel('Exposure (Disparate Impact Normalized)')
    elif title.startswith("plot_disptr_"):
      plt.ylabel('Exposure (Disparate Treatment)')
    else: plt.ylabel('Exposure')
    #this_range = [str(int(id)) + ":" + "{:.{}f}".format(percent,1) + "%" for id, percent in zip(np.arange(min(x2), max(x2)+1, 1.0), percents)]
    plt.xticks(np.arange(y_max, y_min, 0.1))
    plt.yticks(np.arange(y_min, y_max, 0.1))
    axes = plt.gca()
    axes.set_ylim([y_min,y_max])
    axes.yaxis.grid()
    
    plt.savefig(title + '.png')
    upload_file(title + '.png', destination)
    plt.close()

def pipeline3m(business_id, method, df_ranking_by_relevance, df_ranking_by_date, 
              df_ranking_by_random, percents, list_of_attributes,
              alluser, destination, df_groups,product_position,votes,reviewers):
  
  df_result = pd.DataFrame(columns=['Exposure_method', 'Context', 'Means', 'P-value'])
  df_result.to_csv('result.csv')
  id_new_file = upload_file('result.csv', destination)


  print("\n++++++++++++++++ EXPOSURE CALCULATION ++++++++++++++++++\n")
  
  print("----------------DEMOGRAPHIC PARITY EXP------------------\n")
  print("------------Ranking by Amazon filter:------------\n")
  yelp_exposures, yelp_user_exposures = print_demographic_parity_exposure_m(business_id,
                                                    method, df_ranking_by_relevance,
                                                     "yelp_", list_of_attributes, destination,product_position,votes,reviewers)
  print("------------Ranking by Date:------------\n")
  date_exposures, date_user_exposures = print_demographic_parity_exposure_m(business_id,
                                                    method, df_ranking_by_date,
                                                     "date_", list_of_attributes, destination,product_position,votes,reviewers)
  print("------------Ranking Random:------------\n")
  random_exposures, rand_user_exposures = print_demographic_parity_exposure_m(business_id,
                                                      method, df_ranking_by_random,
                                                       "rand_", list_of_attributes, destination,product_position,votes,reviewers)
  
  array_group_id = np.arange(0,df_groups['group_id'].max()+1)
  filePath = stat_significance_inter_rankings(yelp_user_exposures, rand_user_exposures, method,
                                   "demgr_yelp_", business_id, array_group_id, destination) # group_id
  upload_file(filePath, destination)
  filePath = stat_significance_inter_rankings(date_user_exposures, rand_user_exposures, method,
                                   "demgr_date_", business_id, array_group_id, destination)
  upload_file(filePath, destination)

  if alluser and not set(list_of_attributes).intersection(set(['age', 'gender', 'ethnicity'])):
    yelp_error = st.pstdev(yelp_user_exposures['exposure'])
    date_error = st.pstdev(date_user_exposures['exposure'])
    rand_error = st.pstdev(rand_user_exposures['exposure'])
  else:
    yelp_error = st.stdev(yelp_user_exposures['exposure'])
    date_error = st.stdev(date_user_exposures['exposure'])
    rand_error = st.stdev(rand_user_exposures['exposure'])
  
  get_plots(yelp_exposures, date_exposures, random_exposures, percents,
            'plot_demgr_' + method + '_' + business_id, list_of_attributes,
            method, business_id, alluser, [yelp_error, date_error, rand_error], destination)
  get_scatter_plots(yelp_user_exposures, date_user_exposures, random_exposures,
                    'scatter_demgr_' + method + '_' + business_id, list_of_attributes,
                    method, business_id, alluser, destination)
  get_distribution_plots(yelp_user_exposures, date_user_exposures, random_exposures,
                    'distribution_demgr_' + method + '_' + business_id, list_of_attributes,
                    method, business_id, alluser, destination)
  

  print("----------------DISPARATE TREATMENT EXP------------------\n")
  print("------------Ranking by Amazon filter:------------\n")
  yelp_exposures, yelp_user_exposures = print_disparate_treatment_exposure_m(business_id,
                                                    method, df_ranking_by_relevance,
                                                     "yelp_", list_of_attributes, alluser, destination,product_position,votes,reviewers)
  print("------------Ranking by Date:------------\n")
  date_exposures, date_user_exposures = print_disparate_treatment_exposure_m(business_id,
                                                    method, df_ranking_by_date,
                                                     "date_", list_of_attributes, alluser, destination,product_position,votes,reviewers)
  print("------------Ranking Random:------------\n")
  random_exposures, rand_user_exposures = print_disparate_treatment_exposure_m(business_id,
                                                      method, df_ranking_by_random,
                                                       "rand_", list_of_attributes, alluser, destination,product_position,votes,reviewers)
  
  array_group_id = np.arange(0,df_groups['group_id'].max()+1)
  filePath = stat_significance_inter_rankings(yelp_user_exposures, rand_user_exposures, method,
                                   "disptr_yelp_", business_id, array_group_id, destination) # group_id
  upload_file(filePath, destination)
  filePath = stat_significance_inter_rankings(date_user_exposures, rand_user_exposures, method,
                                   "disptr_date_", business_id, array_group_id, destination)
  upload_file(filePath, destination)

  if alluser and not set(list_of_attributes).intersection(set(['age', 'gender', 'ethnicity'])):
    yelp_error = st.pstdev(yelp_user_exposures['exposure'])
    date_error = st.pstdev(date_user_exposures['exposure'])
    rand_error = st.pstdev(rand_user_exposures['exposure'])
  else:
    yelp_error = st.stdev(yelp_user_exposures['exposure'])
    date_error = st.stdev(date_user_exposures['exposure'])
    rand_error = st.stdev(rand_user_exposures['exposure'])
  
  get_plots(yelp_exposures, date_exposures, random_exposures, percents,
            'plot_disptr_' + method + '_' + business_id, list_of_attributes,
            method, business_id, alluser, [yelp_error, date_error, rand_error], destination)
  get_scatter_plots(yelp_user_exposures, date_user_exposures, rand_user_exposures,
                    'scatter_disptr_' + method + '_' + business_id, list_of_attributes,
                    method, business_id, alluser, destination)
  get_distribution_plots(yelp_user_exposures, date_user_exposures, random_exposures,
                    'distribution_disptr_' + method + '_' + business_id, list_of_attributes,
                    method, business_id, alluser, destination)




  print("----------------DISPARATE IMPACT EXP------------------\n")
  print("------------Ranking by Yelp filter:------------\n")
  yelp_exposures, yelp_user_exposures = print_disparate_impact_exposure_m(business_id, method, df_ranking_by_relevance,
                                                   "yelp_", list_of_attributes, alluser, destination,product_position,votes,reviewers)
  
  print("------------Ranking by Date:------------\n")
  date_exposures, date_user_exposures = print_disparate_impact_exposure_m(business_id, method, df_ranking_by_date,
                                                   "date_", list_of_attributes, alluser, destination,product_position,votes,reviewers)
  
  print("------------Ranking Random:------------\n")
  random_exposures, rand_user_exposures = print_disparate_impact_exposure_m(business_id, method, df_ranking_by_random,
                                                     "rand_", list_of_attributes, alluser, destination,product_position,votes,reviewers)
  
  filePath = stat_significance_inter_rankings(yelp_user_exposures, rand_user_exposures, method,
                                   "dispimp_yelp_", business_id, array_group_id, destination)
  upload_file(filePath, destination)
  filePath = stat_significance_inter_rankings(date_user_exposures, rand_user_exposures, method,
                                   "dispimp_date_", business_id, array_group_id, destination)
  upload_file(filePath, destination)

  if alluser and not set(list_of_attributes).intersection(set(['age', 'gender', 'ethnicity'])):
    yelp_error = st.pstdev(yelp_user_exposures['exposure'])
    date_error = st.pstdev(date_user_exposures['exposure'])
    rand_error = st.pstdev(rand_user_exposures['exposure'])
  else:
    yelp_error = st.stdev(yelp_user_exposures['exposure'])
    date_error = st.stdev(date_user_exposures['exposure'])
    rand_error = st.stdev(rand_user_exposures['exposure'])
  
  get_plots(yelp_exposures, date_exposures, random_exposures, percents,
            'plot_dispimp_' + method + '_' + business_id, list_of_attributes,
            method, business_id, alluser, [yelp_error, date_error, rand_error], destination)
  get_scatter_plots(yelp_user_exposures, date_user_exposures, rand_user_exposures,
                    'scatter_dispimp_' + method + '_' + business_id, list_of_attributes,
                    method, business_id, alluser, destination)
  get_distribution_plots(yelp_user_exposures, date_user_exposures, random_exposures,
                    'distribution_dispimp_' + method + '_' + business_id, list_of_attributes,
                    method, business_id, alluser, destination)
#PRINTS  
def print_disparate_impact_exposure_m(business_id, method,  df_ranking,
                                    filename, list_of_attributes, alluser, destination,product_position,votes,reviewers):
  df_groups = pd.read_csv('groups_' + method + '_' + business_id + '.csv')
  i = 0
  exposures = pd.DataFrame(columns=['group_id', 'exposure'])
  user_exposures = pd.DataFrame(columns=['user_id', 'group_id', 'exposure'])
  user_exposures['user_id'] = df_ranking['user_id']
  while i <= df_groups['group_id'].max():
      current_exp, user_exposures = disparate_impact_exposure_m(df_groups[df_groups['group_id'] == i], df_ranking,user_exposures, business_id, i, alluser, product_position,votes,reviewers)
      exposures.loc[i, 'group_id'] = i
      exposures.loc[i, 'exposure'] = current_exp
      i = i + 1
  
  #destination = set_file_destination(list_of_attributes, method, business_id)
  user_exposures.to_csv('user_exp_dispimp_' + method + '_' + filename + business_id + '.csv')
  upload_file('user_exp_dispimp_' + method + '_' + filename + business_id + '.csv',
                  destination)
  
  filePath = stat_significance_inter_groups(df_groups, 'dispimp_'+filename, business_id,
                                            method, user_exposures, destination)

  exposures.to_csv('exp_dispimp_' + method + '_' + filename + business_id + '.csv')
  upload_file('exp_dispimp_' + method + '_' + filename + business_id + '.csv',
                  destination)
  upload_file(filePath, destination)
  print(exposures)
  print('\n')
  return exposures, user_exposures

def print_demographic_parity_exposure_m(business_id, method, df_ranking, filename,
                                      list_of_attributes, destination,product_position,votes,reviewers):
  df_groups = pd.read_csv('groups_' + method + '_' + business_id + '.csv')
  i = 0
  exposures = pd.DataFrame(columns=['group_id', 'exposure'])
  user_exposures = pd.DataFrame(columns=['user_id', 'group_id', 'exposure'])
  print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
  print(df_ranking)
  user_exposures['user_id'] = df_ranking['user_id']
  while i <= df_groups['group_id'].max():
      current_exp, user_exposures = demographic_parity_exposure_m(df_groups[df_groups['group_id'] == i], df_ranking, user_exposures, i, product_position,votes,reviewers)
      exposures.loc[i, 'group_id'] = i
      exposures.loc[i, 'exposure'] = current_exp
      i = i + 1
  
  #destination = set_file_destination(list_of_attributes, method, business_id)
  user_exposures.to_csv('user_exp_demgr_' + method + '_' + filename + business_id + '.csv')
  upload_file('user_exp_demgr_' + method + '_' + filename + business_id + '.csv',
                  destination)
  filePath = stat_significance_inter_groups(df_groups, 'demgr_'+filename, business_id,
                                            method, user_exposures, destination)

  exposures.to_csv('exp_demgr_' + method + '_' + filename + business_id + '.csv')
  upload_file('exp_demgr_' + method + '_' + filename + business_id + '.csv',
                  destination)
  upload_file(filePath, destination)
  print(exposures)
  print('\n')
  return exposures, user_exposures
  
def print_disparate_treatment_exposure_m(business_id, method, df_ranking, filename,
                                      list_of_attributes, alluser, destination,product_position,votes,reviewers):
  df_groups = pd.read_csv('groups_' + method + '_' + business_id + '.csv')
  i = 0
  exposures = pd.DataFrame(columns=['group_id', 'exposure'])
  user_exposures = pd.DataFrame(columns=['user_id', 'group_id', 'exposure'])
  user_exposures['user_id'] = df_ranking['user_id']
  while i <= df_groups['group_id'].max():
      current_exp, user_exposures = disparate_treatment_exposure_m(df_groups[df_groups['group_id'] == i], df_ranking,user_exposures, business_id, i, alluser, product_position,votes,reviewers)
      exposures.loc[i, 'group_id'] = i
      exposures.loc[i, 'exposure'] = current_exp
      i = i + 1
  
  #destination = set_file_destination(list_of_attributes, method, business_id)
  user_exposures.to_csv('user_exp_disptr_' + method + '_' + filename + business_id + '.csv')
  upload_file('user_exp_disptr_' + method + '_' + filename + business_id + '.csv',
                  destination)
  filePath = stat_significance_inter_groups(df_groups, 'disptr_'+filename, business_id,
                                            method, user_exposures, destination)
  exposures.to_csv('exp_disptr_' + method + '_' + filename + business_id + '.csv')
  upload_file('exp_disptr_' + method + '_' + filename + business_id + '.csv',
                  destination)
  upload_file(filePath, destination)
  print(exposures)
  print('\n')
  return exposures, user_exposures
#EXPOSURES
def disparate_impact_exposure_m(df_group, ranking, user_exposures, business_id, group_index, alluser,product_position,votes,reviewers):
  primo = ranking['useful_votes'].values.tolist()
  norm = 1
  for x in primo:
    temp = math.log(x+2, 2)
    if temp > norm:
      norm = temp

  all_exposures = []
  for i, user in df_group.iterrows():
    user_id = user['user_id']

    if alluser:
      useful = ranking[ranking['user_id'] == user_id]['useful_votes'].values[0]

    
    counts = useful + 2
    
    base = 2  # con 10 i valori sono troppo bassi
    counts = math.log(counts, base)
    #counts = counts/norm
    
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print(df_group)
    print(ranking)
    position = ranking[ranking['user_id'] == user_id]['position'].values[0]
    
    current_exp = exp(position) * counts * ((votes[product_position-1]*reviewers[product_position-1])/max(reviewers))
    
    all_exposures.append(current_exp)
    user_exposures.loc[user_exposures['user_id'] == user_id, 'exposure'] = current_exp
    user_exposures.loc[user_exposures['user_id'] == user_id, 'group_id'] = group_index
  print('all_exposures size:', len(all_exposures))
  mean = st.mean(all_exposures)
  return mean, user_exposures


def demographic_parity_exposure_m(df_group, ranking, user_exposures, group_index,product_position,votes,reviewers):
  #(stellette*n.review)/(n.max di review per un item)
  all_exposures = []
  for i, user in df_group.iterrows():
      user_id = user['user_id']
      position = ranking[ranking['user_id'] == user_id]['position'].values[0]
      current_exp = exp(position) * ((votes[product_position-1]*reviewers[product_position-1])/max(reviewers))
      all_exposures.append(current_exp)
      user_exposures.loc[user_exposures['user_id'] == user_id, 'exposure'] = current_exp
      user_exposures.loc[user_exposures['user_id'] == user_id, 'group_id'] = group_index
  print('all_exposures size:', len(all_exposures))
  mean = st.mean(all_exposures)
  return mean, user_exposures

def disparate_treatment_exposure_m(df_group, ranking, user_exposures, business_id, group_index, alluser,product_position,votes,reviewers):
  all_exposures = []
  for i, user in df_group.iterrows():
    user_id = user['user_id']

    if alluser:
      useful = ranking[ranking['user_id'] == user_id]['useful_votes'].values[0]
    
    counts = useful + 2
    
    base = 2  # con 10 i valori sono troppo bassi
    counts = math.log(counts, base)
    
    position = ranking[ranking['user_id'] == user_id]['position'].values[0]
    
    current_exp = exp(position) * ((votes[product_position-1]*reviewers[product_position-1])/max(reviewers)) / counts 
    
    all_exposures.append(current_exp)
    user_exposures.loc[user_exposures['user_id'] == user_id, 'exposure'] = current_exp
    user_exposures.loc[user_exposures['user_id'] == user_id, 'group_id'] = group_index
  print('all_exposures size:', len(all_exposures))
  mean = st.mean(all_exposures)
  return mean, user_exposures

def normalize_exposure(df_exposure):
  normazized_df_exposure = pd.DataFrame(columns=['user_id','group_id','exposure'])
  normalizing_factor=max(df_exposure['exposure'].tolist())#for MAX
  #normalizing_factor=np.linalg.norm(df_exposure['exposure'].tolist())# for EUCLIDEAN NORM
  for i, elem in df_exposure.iterrows():
    row = {
        'user_id': elem['user_id'],
        'group_id': elem['group_id'],
        'exposure': float(elem['exposure'])/normalizing_factor
    }
    normazized_df_exposure = normazized_df_exposure.append(row, ignore_index=True)
  return normazized_df_exposure

def mean_normalized_exposure(df_exposure):
  #df_groups = pd.read_csv('groups_' + method + '_' + business_id + '.csv')
  i = 0
  exposures = pd.DataFrame(columns=['group_id', 'exposure'])
  #user_exposures = pd.DataFrame(columns=['user_id', 'group_id', 'exposure'])
  #user_exposures['user_id'] = df_exposure['user_id']
  while i <= df_exposure['group_id'].max():
      current_exp = st.mean(df_exposure[df_exposure['group_id'] == i]['exposure'].values.tolist())
      exposures.loc[i, 'group_id'] = i
      exposures.loc[i, 'exposure'] = current_exp
      i = i + 1

  return exposures

In [ ]:
#ESECUZIONE PIPELINE SENZA ASSUNZIONI
drive = authenticate()
download = drive.CreateFile({'id': '1V-L7IqaPVcBp4RvWKnUXt00SHrsT1e9O'})
download.GetContentFile('queries.csv')

queries = pd.read_csv("queries.csv")
queries = drop_unnamed(queries)

query=4
products_gtree_id = queries.loc[queries["cardinal"]==query,"products_gtree"].tolist()[0]
products_folder_id = queries.loc[queries["cardinal"]==query,"products_folder"].tolist()[0]
reviews_folder_id = queries.loc[queries["cardinal"]==query,"reviews_folder"].tolist()[0]
fairness_data_id = queries.loc[queries["cardinal"]==query,"fairness_data"].tolist()[0]
face_data_id = queries.loc[queries["cardinal"]==query,"face_data"].tolist()[0]

download = drive.CreateFile({'id': products_gtree_id})
download.GetContentFile(str(query)+'_gtree.csv')

gtree = pd.read_csv(str(query)+'_gtree.csv')
gtree = drop_unnamed(gtree)

products_id_list = ["B01LXY19XD"]


#list_of_attributes = ['review_count'] # alluser
#list_of_attributes = ['review_count', 'fans', 'average_stars', 'useful', 'funny', 'cool'] # dataset
#list_of_attributes = ['loc1', 'loc2', 'loc3'] # dataset
#list_of_attributes = ['age', 'gender', 'ethnicity'] # alluser
#list_of_attributes = ['fans'] # dataset
#list_of_attributes = ['useful', 'funny', 'cool'] # dataset
#list_of_attributes = ['gender', 'ethnicity'] # alluser
#list_of_attributes = ['ethnicity'] # alluser
#list_of_attributes = ['review_count', 'friend_count', 'photo_count'] # alluser
#list_of_attributes = ['review_sentiment'] # alluser
list_of_attributes = ['gender'] # alluser
method = 'custom'
folder_name = '#14_custom_gender_'
gtree_folder_destination_name = 'gfolder_#14'
N_of_groups = 3
alluser = True

for id in products_id_list:
  print(id)
  drive = authenticate()
  df_rel_ranking, df_date_ranking, df_rand_ranking = pipeline1(id, alluser) 
  
  destination = create_folder_in_drive(drive, folder_name+str(id), products_folder_id)
  destination_copy = destination

  group_list, percents, destination = pipeline2(df_rel_ranking, id,
                                  N_of_groups, list_of_attributes, method,
                                  alluser, destination, '')
  
  pipeline3(id, method, df_rel_ranking, df_date_ranking, df_rand_ranking,
            percents, list_of_attributes, alluser, destination, group_list)
print("END")

In [ ]:
#GENERA GRAFICI DI DISTRIBUZIONE CON TAGLI 0.05
drive = authenticate()
id_item = "B01LXY19XD"
example_name = "Amazon_4_5_"+ id_item +"_taglio5"
percentuale_taglio = 0.05

width = 0.20
y_min = 0.0
y_max = 1.5
ticks = 0.1

def taglia(lista,percentuale):
  temp = []
  point = None
  for elem in lista:
    if point is None:
      point = elem
      temp.append(point)
    if point is not None:
      if ((point-elem)/point) > percentuale:
        point = elem
        temp.append(point)
    
  return temp

example_folder = create_folder_in_drive(drive, example_name, "15NfEUQv3U5ARe7jBPzf4SrAx8SN-vTbO")

demog_folder = create_folder_in_drive(drive, "Demographic parity", example_folder)
dispimp_folder = create_folder_in_drive(drive, "Disparate impact", example_folder)
disptr_folder = create_folder_in_drive(drive, "Disparate treatment", example_folder)

all_demog_folder = create_folder_in_drive(drive, "All_rankings", demog_folder)
amazon_demog_folder = create_folder_in_drive(drive, "amazon_rankings", demog_folder)
date_demog_folder = create_folder_in_drive(drive, "date_rankings", demog_folder)
rand_demog_folder = create_folder_in_drive(drive, "random_rankings", demog_folder)

all_dispimp_folder = create_folder_in_drive(drive, "All_rankings", dispimp_folder)
amazon_dispimp_folder = create_folder_in_drive(drive, "amazon_rankings", dispimp_folder)
date_dispimp_folder = create_folder_in_drive(drive, "date_rankings", dispimp_folder)
rand_dispimp_folder = create_folder_in_drive(drive, "random_rankings", dispimp_folder)

all_disptr_folder = create_folder_in_drive(drive, "All_rankings", disptr_folder)
amazon_disptr_folder = create_folder_in_drive(drive, "amazon_rankings", disptr_folder)
date_disptr_folder = create_folder_in_drive(drive, "date_rankings", disptr_folder)
rand_disptr_folder = create_folder_in_drive(drive, "random_rankings", disptr_folder)

###   DEMOG   ###

df = pd.read_csv('user_exp_demgr_custom_yelp_'+ id_item +'.csv')
df_date = pd.read_csv('user_exp_demgr_custom_date_'+ id_item +'.csv')
df_random = pd.read_csv('user_exp_demgr_custom_rand_'+ id_item +'.csv')

x1=[]
x2=[]
x3=[]

x1_date=[]
x2_date=[]
x3_date=[]

x1_random=[]
x2_random=[]
x3_random=[]

for i, el in df.iterrows():
  if el['group_id'] == 0:
    x1.append(el['exposure'])

for i, el in df.iterrows():
  if el['group_id'] == 1:
    x2.append(el['exposure'])

for i, el in df.iterrows():
  if el['group_id'] == 2:
    x3.append(el['exposure'])
    
for i, el in df_date.iterrows():
  if el['group_id'] == 0:
    x1_date.append(el['exposure'])

for i, el in df_date.iterrows():
  if el['group_id'] == 1:
    x2_date.append(el['exposure'])

for i, el in df_date.iterrows():
  if el['group_id'] == 2:
    x3_date.append(el['exposure'])

for i, el in df_random.iterrows():
  if el['group_id'] == 0:
    x1_random.append(el['exposure'])

for i, el in df_random.iterrows():
  if el['group_id'] == 1:
    x2_random.append(el['exposure'])

for i, el in df_random.iterrows():
  if el['group_id'] == 2:
    x3_random.append(el['exposure'])

x1.sort(reverse=True)
x2.sort(reverse=True)
x3.sort(reverse=True)

x1_date.sort(reverse=True)
x2_date.sort(reverse=True)
x3_date.sort(reverse=True)

x1_random.sort(reverse=True)
x2_random.sort(reverse=True)
x3_random.sort(reverse=True)


if percentuale_taglio is not 0:
  x1 = taglia(x1,percentuale_taglio)
  x2 = taglia(x2,percentuale_taglio)
  x3 = taglia(x3,percentuale_taglio)

  x1_date = taglia(x1_date,percentuale_taglio)
  x2_date = taglia(x2_date,percentuale_taglio)
  x3_date = taglia(x3_date,percentuale_taglio)

  x1_random = taglia(x1_random,percentuale_taglio)
  x2_random = taglia(x2_random,percentuale_taglio)
  x3_random = taglia(x3_random,percentuale_taglio)

min_lenght = len(x1)
if len(x2) < min_lenght:
  min_lenght = len(x2)
if len(x3) < min_lenght:
  min_lenght = len(x3)

x1 = x1[:min_lenght]
x2 = x2[:min_lenght]
x3 = x3[:min_lenght]

sum_x1 = sum(x1)
sum_x2 = sum(x2)
sum_x3 = sum(x3)

min_lenght_date = len(x1_date)
if len(x2_date) < min_lenght_date:
  min_lenght_date = len(x2_date)
if len(x3_date) < min_lenght_date:
  min_lenght_date = len(x3_date)

x1_date = x1_date[:min_lenght_date]
x2_date = x2_date[:min_lenght_date]
x3_date = x3_date[:min_lenght_date]

sum_x1_date = sum(x1_date)
sum_x2_date = sum(x2_date)
sum_x3_date = sum(x3_date)

min_lenght_random = len(x1_random)
if len(x2_random) < min_lenght_random:
  min_lenght_random = len(x2_random)
if len(x3_random) < min_lenght_random:
  min_lenght_random = len(x3_random)

x1_random = x1_random[:min_lenght_random]
x2_random = x2_random[:min_lenght_random]
x3_random = x3_random[:min_lenght_random]

sum_x1_random = sum(x1_random)
sum_x2_random = sum(x2_random)
sum_x3_random = sum(x3_random)

print("### 1 ###")

### AMAZON ###

plt.scatter(list(range(len(x1))),x1, s=10, color='red', label='unknowns')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Demographic Parity)')
plt.xlabel('Exp sum: ' + str(sum_x1))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('0-unknowns.png')
upload_file('0-unknowns.png', amazon_demog_folder)
plt.close()

plt.scatter(list(range(len(x2))),x2, s=10, color='green', label='woman')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Demographic Parity)')
plt.xlabel('Exp sum: ' + str(sum_x2))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('1-woman.png')
upload_file('1-woman.png', amazon_demog_folder)
plt.close()

plt.scatter(list(range(len(x3))),x3, s=10, color='blue', label='man')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Demographic Parity)')
plt.xlabel('Exp sum: ' + str(sum_x3))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('2-man.png')
upload_file('2-man.png', amazon_demog_folder)
plt.close()

plt.scatter(list(range(len(x1))),x1, s=10, color='red', label='unknowns')
plt.scatter(list(range(len(x2))),x2, s=10, color='green', label='woman')
plt.scatter(list(range(len(x3))),x3, s=10, color='blue', label='man')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Demographic Parity)')
plt.xlabel('Exp sum: u: ' + str(sum_x1) + ' f: ' + str(sum_x2) + ' m: ' + str(sum_x3))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('All.png')
upload_file('All.png', amazon_demog_folder)
plt.close()

### DATE ###

plt.scatter(list(range(len(x1_date))),x1_date, s=10, color='red', label='unknowns')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Demographic Parity)')
plt.xlabel('Exp sum: ' + str(sum_x1_date))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('0-unknowns-date.png')
upload_file('0-unknowns-date.png', date_demog_folder)
plt.close()

plt.scatter(list(range(len(x2_date))),x2_date, s=10, color='green', label='woman')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Demographic Parity)')
plt.xlabel('Exp sum: ' + str(sum_x2_date))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('1-woman-date.png')
upload_file('1-woman-date.png', date_demog_folder)
plt.close()

plt.scatter(list(range(len(x3_date))),x3_date, s=10, color='blue', label='man')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Demographic Parity)')
plt.xlabel('Exp sum: ' + str(sum_x3_date))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('2-man-date.png')
upload_file('2-man-date.png', date_demog_folder)
plt.close()

plt.scatter(list(range(len(x1_date))),x1_date, s=10, color='red', label='unknowns')
plt.scatter(list(range(len(x2_date))),x2_date, s=10, color='green', label='woman')
plt.scatter(list(range(len(x3_date))),x3_date, s=10, color='blue', label='man')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Demographic Parity)')
plt.xlabel('Exp sum: u: ' + str(sum_x1_date) + ' f: ' + str(sum_x2_date) + ' m: ' + str(sum_x3_date))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('All-date.png')
upload_file('All-date.png', date_demog_folder)
plt.close()

### RANDOM ###

plt.scatter(list(range(len(x1_random))),x1_random, s=10, color='red', label='unknowns')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Demographic Parity)')
plt.xlabel('Exp sum: ' + str(sum_x1_random))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('0-unknowns-random.png')
upload_file('0-unknowns-random.png', rand_demog_folder)
plt.close()

plt.scatter(list(range(len(x2_random))),x2_random, s=10, color='green', label='woman')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Demographic Parity)')
plt.xlabel('Exp sum: ' + str(sum_x2_random))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('1-woman-random.png')
upload_file('1-woman-random.png', rand_demog_folder)
plt.close()

plt.scatter(list(range(len(x3_random))),x3_random, s=10, color='blue', label='man')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Demographic Parity)')
plt.xlabel('Exp sum: ' + str(sum_x3_random))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('2-man-random.png')
upload_file('2-man-random.png', rand_demog_folder)
plt.close()

plt.scatter(list(range(len(x1_random))),x1_random, s=10, color='red', label='unknowns')
plt.scatter(list(range(len(x2_random))),x2_random, s=10, color='green', label='woman')
plt.scatter(list(range(len(x3_random))),x3_random, s=10, color='blue', label='man')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Demographic Parity)')
plt.xlabel('Exp sum: u: ' + str(sum_x1_random) + ' f: ' + str(sum_x2_random) + ' m: ' + str(sum_x3_random))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('All-random.png')
upload_file('All-random.png', rand_demog_folder)
plt.close()

### ALL ###

min_lenght = len(x1)
if len(x1_date) < min_lenght:
  min_lenght = len(x1_date)
if len(x1_random) < min_lenght:
  min_lenght = len(x1_random)
if len(x2) < min_lenght:
  min_lenght = len(x2)
if len(x2_date) < min_lenght:
  min_lenght = len(x2_date)
if len(x2_random) < min_lenght:
  min_lenght = len(x2_random)
if len(x3) < min_lenght:
  min_lenght = len(x3)
if len(x3_date) < min_lenght:
  min_lenght = len(x3_date)
if len(x3_random) < min_lenght:
  min_lenght = len(x3_random)

print("### PRE ALL ###")

x1 = x1[:min_lenght]
x1_date = x1_date[:min_lenght]
x1_random = x1_random[:min_lenght]
x2 = x2[:min_lenght]
x2_date = x2_date[:min_lenght]
x2_random = x2_random[:min_lenght]
x3 = x3[:min_lenght]
x3_date = x3_date[:min_lenght]
x3_random = x3_random[:min_lenght]

sum_x1 = sum(x1)
sum_x1_date = sum(x1_date)
sum_x1_random = sum(x1_random)
sum_x2 = sum(x2)
sum_x2_date = sum(x2_date)
sum_x2_random = sum(x2_random)
sum_x3 = sum(x3)
sum_x3_date = sum(x3_date)
sum_x3_random = sum(x3_random)

print("### ALL ###")

plt.scatter(list(range(len(x1))),x1, s=10, color='red', label='amazon')
plt.scatter(list(range(len(x1_date))),x1_date, s=10, color='green', label='date')
plt.scatter(list(range(len(x1_random))),x1_random, s=10, color='blue', label='random')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Demographic Parity)')
plt.xlabel('Exp sum: amazon: ' + str(sum_x1) + ' date: ' + str(sum_x1_date) + ' rand: ' + str(sum_x1_random))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('0-unknowns-allranks.png')
upload_file('0-unknowns-allranks.png', all_demog_folder)
plt.close()

plt.scatter(list(range(len(x2))),x2, s=10, color='red', label='amazon')
plt.scatter(list(range(len(x2_date))),x2_date, s=10, color='green', label='date')
plt.scatter(list(range(len(x2_random))),x2_random, s=10, color='blue', label='random')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Demographic Parity)')
plt.xlabel('Exp sum: amazon: ' + str(sum_x2) + ' date: ' + str(sum_x2_date) + ' rand: ' + str(sum_x2_random))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('1-woman-allranks.png')
upload_file('1-woman-allranks.png', all_demog_folder)
plt.close()

plt.scatter(list(range(len(x3))),x3, s=10, color='red', label='amazon')
plt.scatter(list(range(len(x3_date))),x3_date, s=10, color='green', label='date')
plt.scatter(list(range(len(x3_random))),x3_random, s=10, color='blue', label='random')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Demographic Parity)')
plt.xlabel('Exp sum: amazon: ' + str(sum_x3) + ' date: ' + str(sum_x3_date) + ' rand: ' + str(sum_x3_random))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('2-man-allranks.png')
upload_file('2-man-allranks.png', all_demog_folder)
plt.close()

###   DISPIMP   ###

width = 0.20
y_min = 0.0
y_max = 12
ticks = 1

df = pd.read_csv('user_exp_dispimp_custom_yelp_'+ id_item +'.csv')
df_date = pd.read_csv('user_exp_dispimp_custom_date_'+ id_item +'.csv')
df_random = pd.read_csv('user_exp_dispimp_custom_rand_'+ id_item +'.csv')

x1=[]
x2=[]
x3=[]

x1_date=[]
x2_date=[]
x3_date=[]

x1_random=[]
x2_random=[]
x3_random=[]

for i, el in df.iterrows():
  if el['group_id'] == 0:
    x1.append(el['exposure'])

for i, el in df.iterrows():
  if el['group_id'] == 1:
    x2.append(el['exposure'])

for i, el in df.iterrows():
  if el['group_id'] == 2:
    x3.append(el['exposure'])
    
for i, el in df_date.iterrows():
  if el['group_id'] == 0:
    x1_date.append(el['exposure'])

for i, el in df_date.iterrows():
  if el['group_id'] == 1:
    x2_date.append(el['exposure'])

for i, el in df_date.iterrows():
  if el['group_id'] == 2:
    x3_date.append(el['exposure'])

for i, el in df_random.iterrows():
  if el['group_id'] == 0:
    x1_random.append(el['exposure'])

for i, el in df_random.iterrows():
  if el['group_id'] == 1:
    x2_random.append(el['exposure'])

for i, el in df_random.iterrows():
  if el['group_id'] == 2:
    x3_random.append(el['exposure'])

x1.sort(reverse=True)
x2.sort(reverse=True)
x3.sort(reverse=True)

x1_date.sort(reverse=True)
x2_date.sort(reverse=True)
x3_date.sort(reverse=True)

x1_random.sort(reverse=True)
x2_random.sort(reverse=True)
x3_random.sort(reverse=True)


if percentuale_taglio is not 0:
  x1 = taglia(x1,percentuale_taglio)
  x2 = taglia(x2,percentuale_taglio)
  x3 = taglia(x3,percentuale_taglio)

  x1_date = taglia(x1_date,percentuale_taglio)
  x2_date = taglia(x2_date,percentuale_taglio)
  x3_date = taglia(x3_date,percentuale_taglio)

  x1_random = taglia(x1_random,percentuale_taglio)
  x2_random = taglia(x2_random,percentuale_taglio)
  x3_random = taglia(x3_random,percentuale_taglio)

min_lenght = len(x1)
if len(x2) < min_lenght:
  min_lenght = len(x2)
if len(x3) < min_lenght:
  min_lenght = len(x3)

x1 = x1[:min_lenght]
x2 = x2[:min_lenght]
x3 = x3[:min_lenght]

sum_x1 = sum(x1)
sum_x2 = sum(x2)
sum_x3 = sum(x3)

min_lenght_date = len(x1_date)
if len(x2_date) < min_lenght_date:
  min_lenght_date = len(x2_date)
if len(x3_date) < min_lenght_date:
  min_lenght_date = len(x3_date)

x1_date = x1_date[:min_lenght_date]
x2_date = x2_date[:min_lenght_date]
x3_date = x3_date[:min_lenght_date]

sum_x1_date = sum(x1_date)
sum_x2_date = sum(x2_date)
sum_x3_date = sum(x3_date)

min_lenght_random = len(x1_random)
if len(x2_random) < min_lenght_random:
  min_lenght_random = len(x2_random)
if len(x3_random) < min_lenght_random:
  min_lenght_random = len(x3_random)

x1_random = x1_random[:min_lenght_random]
x2_random = x2_random[:min_lenght_random]
x3_random = x3_random[:min_lenght_random]

sum_x1_random = sum(x1_random)
sum_x2_random = sum(x2_random)
sum_x3_random = sum(x3_random)

print("### 2 ###")

### AMAZON ###

plt.scatter(list(range(len(x1))),x1, s=10, color='red', label='unknowns')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Impact)')
plt.xlabel('Exp sum: ' + str(sum_x1))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('0-unknowns.png')
upload_file('0-unknowns.png', amazon_dispimp_folder)
plt.close()

plt.scatter(list(range(len(x2))),x2, s=10, color='green', label='woman')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Impact)')
plt.xlabel('Exp sum: ' + str(sum_x2))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('1-woman.png')
upload_file('1-woman.png', amazon_dispimp_folder)
plt.close()

plt.scatter(list(range(len(x3))),x3, s=10, color='blue', label='man')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Impact)')
plt.xlabel('Exp sum: ' + str(sum_x3))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('2-man.png')
upload_file('2-man.png', amazon_dispimp_folder)
plt.close()

plt.scatter(list(range(len(x1))),x1, s=10, color='red', label='unknowns')
plt.scatter(list(range(len(x2))),x2, s=10, color='green', label='woman')
plt.scatter(list(range(len(x3))),x3, s=10, color='blue', label='man')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Impact)')
plt.xlabel('Exp sum: u: ' + str(sum_x1) + ' f: ' + str(sum_x2) + ' m: ' + str(sum_x3))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('All.png')
upload_file('All.png', amazon_dispimp_folder)
plt.close()

### DATE ###

plt.scatter(list(range(len(x1_date))),x1_date, s=10, color='red', label='unknowns')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Impact)')
plt.xlabel('Exp sum: ' + str(sum_x1_date))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('0-unknowns-date.png')
upload_file('0-unknowns-date.png', date_dispimp_folder)
plt.close()

plt.scatter(list(range(len(x2_date))),x2_date, s=10, color='green', label='woman')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Impact)')
plt.xlabel('Exp sum: ' + str(sum_x2_date))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('1-woman-date.png')
upload_file('1-woman-date.png', date_dispimp_folder)
plt.close()

plt.scatter(list(range(len(x3_date))),x3_date, s=10, color='blue', label='man')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Impact)')
plt.xlabel('Exp sum: ' + str(sum_x3_date))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('2-man-date.png')
upload_file('2-man-date.png', date_dispimp_folder)
plt.close()

plt.scatter(list(range(len(x1_date))),x1_date, s=10, color='red', label='unknowns')
plt.scatter(list(range(len(x2_date))),x2_date, s=10, color='green', label='woman')
plt.scatter(list(range(len(x3_date))),x3_date, s=10, color='blue', label='man')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Impact)')
plt.xlabel('Exp sum: u: ' + str(sum_x1_date) + ' f: ' + str(sum_x2_date) + ' m: ' + str(sum_x3_date))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('All-date.png')
upload_file('All-date.png', date_dispimp_folder)
plt.close()

### RANDOM ###

plt.scatter(list(range(len(x1_random))),x1_random, s=10, color='red', label='unknowns')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Impact)')
plt.xlabel('Exp sum: ' + str(sum_x1_random))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('0-unknowns-random.png')
upload_file('0-unknowns-random.png', rand_dispimp_folder)
plt.close()

plt.scatter(list(range(len(x2_random))),x2_random, s=10, color='green', label='woman')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Impact)')
plt.xlabel('Exp sum: ' + str(sum_x2_random))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('1-woman-random.png')
upload_file('1-woman-random.png', rand_dispimp_folder)
plt.close()

plt.scatter(list(range(len(x3_random))),x3_random, s=10, color='blue', label='man')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Impact)')
plt.xlabel('Exp sum: ' + str(sum_x3_random))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('2-man-random.png')
upload_file('2-man-random.png', rand_dispimp_folder)
plt.close()

plt.scatter(list(range(len(x1_random))),x1_random, s=10, color='red', label='unknowns')
plt.scatter(list(range(len(x2_random))),x2_random, s=10, color='green', label='woman')
plt.scatter(list(range(len(x3_random))),x3_random, s=10, color='blue', label='man')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Impact)')
plt.xlabel('Exp sum: u: ' + str(sum_x1_random) + ' f: ' + str(sum_x2_random) + ' m: ' + str(sum_x3_random))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('All-random.png')
upload_file('All-random.png', rand_dispimp_folder)
plt.close()

### ALL ###

min_lenght = len(x1)
if len(x1_date) < min_lenght:
  min_lenght = len(x1_date)
if len(x1_random) < min_lenght:
  min_lenght = len(x1_random)
if len(x2) < min_lenght:
  min_lenght = len(x2)
if len(x2_date) < min_lenght:
  min_lenght = len(x2_date)
if len(x2_random) < min_lenght:
  min_lenght = len(x2_random)
if len(x3) < min_lenght:
  min_lenght = len(x3)
if len(x3_date) < min_lenght:
  min_lenght = len(x3_date)
if len(x3_random) < min_lenght:
  min_lenght = len(x3_random)
print("### PRE ALL ###")

x1 = x1[:min_lenght]
x1_date = x1_date[:min_lenght]
x1_random = x1_random[:min_lenght]
x2 = x2[:min_lenght]
x2_date = x2_date[:min_lenght]
x2_random = x2_random[:min_lenght]
x3 = x3[:min_lenght]
x3_date = x3_date[:min_lenght]
x3_random = x3_random[:min_lenght]

sum_x1 = sum(x1)
sum_x1_date = sum(x1_date)
sum_x1_random = sum(x1_random)
sum_x2 = sum(x2)
sum_x2_date = sum(x2_date)
sum_x2_random = sum(x2_random)
sum_x3 = sum(x3)
sum_x3_date = sum(x3_date)
sum_x3_random = sum(x3_random)

print("### ALL ###")

plt.scatter(list(range(len(x1))),x1, s=10, color='red', label='amazon')
plt.scatter(list(range(len(x1_date))),x1_date, s=10, color='green', label='date')
plt.scatter(list(range(len(x1_random))),x1_random, s=10, color='blue', label='random')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Impact)')
plt.xlabel('Exp sum: amazon: ' + str(sum_x1) + ' date: ' + str(sum_x1_date) + ' rand: ' + str(sum_x1_random))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('0-unknowns-allranks.png')
upload_file('0-unknowns-allranks.png', all_dispimp_folder)
plt.close()

plt.scatter(list(range(len(x2))),x2, s=10, color='red', label='amazon')
plt.scatter(list(range(len(x2_date))),x2_date, s=10, color='green', label='date')
plt.scatter(list(range(len(x2_random))),x2_random, s=10, color='blue', label='random')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Impact)')
plt.xlabel('Exp sum: amazon: ' + str(sum_x2) + ' date: ' + str(sum_x2_date) + ' rand: ' + str(sum_x2_random))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('1-woman-allranks.png')
upload_file('1-woman-allranks.png', all_dispimp_folder)
plt.close()

plt.scatter(list(range(len(x3))),x3, s=10, color='red', label='amazon')
plt.scatter(list(range(len(x3_date))),x3_date, s=10, color='green', label='date')
plt.scatter(list(range(len(x3_random))),x3_random, s=10, color='blue', label='random')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Impact)')
plt.xlabel('Exp sum: amazon: ' + str(sum_x3) + ' date: ' + str(sum_x3_date) + ' rand: ' + str(sum_x3_random))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('2-man-allranks.png')
upload_file('2-man-allranks.png', all_dispimp_folder)
plt.close()

###   DISP TR   ###

width = 0.20
y_min = 0.0
y_max = 1.5
ticks = 0.1

df = pd.read_csv('user_exp_disptr_custom_yelp_'+ id_item +'.csv')
df_date = pd.read_csv('user_exp_disptr_custom_date_'+ id_item +'.csv')
df_random = pd.read_csv('user_exp_disptr_custom_rand_'+ id_item +'.csv')

x1=[]
x2=[]
x3=[]

x1_date=[]
x2_date=[]
x3_date=[]

x1_random=[]
x2_random=[]
x3_random=[]

all_df = df
all_df_date = df_date
all_df_random = df_random

for i, el in df.iterrows():
  if el['group_id'] == 0:
    x1.append(el['exposure'])

for i, el in df.iterrows():
  if el['group_id'] == 1:
    x2.append(el['exposure'])

for i, el in df.iterrows():
  if el['group_id'] == 2:
    x3.append(el['exposure'])
    
for i, el in df_date.iterrows():
  if el['group_id'] == 0:
    x1_date.append(el['exposure'])

for i, el in df_date.iterrows():
  if el['group_id'] == 1:
    x2_date.append(el['exposure'])

for i, el in df_date.iterrows():
  if el['group_id'] == 2:
    x3_date.append(el['exposure'])

for i, el in df_random.iterrows():
  if el['group_id'] == 0:
    x1_random.append(el['exposure'])

for i, el in df_random.iterrows():
  if el['group_id'] == 1:
    x2_random.append(el['exposure'])

for i, el in df_random.iterrows():
  if el['group_id'] == 2:
    x3_random.append(el['exposure'])

x1.sort(reverse=True)
x2.sort(reverse=True)
x3.sort(reverse=True)

x1_date.sort(reverse=True)
x2_date.sort(reverse=True)
x3_date.sort(reverse=True)

x1_random.sort(reverse=True)
x2_random.sort(reverse=True)
x3_random.sort(reverse=True)


if percentuale_taglio is not 0:
  x1 = taglia(x1,percentuale_taglio)
  x2 = taglia(x2,percentuale_taglio)
  x3 = taglia(x3,percentuale_taglio)

  x1_date = taglia(x1_date,percentuale_taglio)
  x2_date = taglia(x2_date,percentuale_taglio)
  x3_date = taglia(x3_date,percentuale_taglio)

  x1_random = taglia(x1_random,percentuale_taglio)
  x2_random = taglia(x2_random,percentuale_taglio)
  x3_random = taglia(x3_random,percentuale_taglio)

min_lenght = len(x1)
if len(x2) < min_lenght:
  min_lenght = len(x2)
if len(x3) < min_lenght:
  min_lenght = len(x3)

x1 = x1[:min_lenght]
x2 = x2[:min_lenght]
x3 = x3[:min_lenght]

sum_x1 = sum(x1)
sum_x2 = sum(x2)
sum_x3 = sum(x3)

min_lenght_date = len(x1_date)
if len(x2_date) < min_lenght_date:
  min_lenght_date = len(x2_date)
if len(x3_date) < min_lenght_date:
  min_lenght_date = len(x3_date)

x1_date = x1_date[:min_lenght_date]
x2_date = x2_date[:min_lenght_date]
x3_date = x3_date[:min_lenght_date]

sum_x1_date = sum(x1_date)
sum_x2_date = sum(x2_date)
sum_x3_date = sum(x3_date)

min_lenght_random = len(x1_random)
if len(x2_random) < min_lenght_random:
  min_lenght_random = len(x2_random)
if len(x3_random) < min_lenght_random:
  min_lenght_random = len(x3_random)

x1_random = x1_random[:min_lenght_random]
x2_random = x2_random[:min_lenght_random]
x3_random = x3_random[:min_lenght_random]

sum_x1_random = sum(x1_random)
sum_x2_random = sum(x2_random)
sum_x3_random = sum(x3_random)

### AMAZON ###

plt.scatter(list(range(len(x1))),x1, s=10, color='red', label='unknowns')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Treatment)')
plt.xlabel('Exp sum: ' + str(sum_x1))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('0-unknowns.png')
upload_file('0-unknowns.png', amazon_disptr_folder)
plt.close()

plt.scatter(list(range(len(x2))),x2, s=10, color='green', label='woman')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Treatment)')
plt.xlabel('Exp sum: ' + str(sum_x2))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('1-woman.png')
upload_file('1-woman.png', amazon_disptr_folder)
plt.close()

plt.scatter(list(range(len(x3))),x3, s=10, color='blue', label='man')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Treatment)')
plt.xlabel('Exp sum: ' + str(sum_x3))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('2-man.png')
upload_file('2-man.png', amazon_disptr_folder)
plt.close()

plt.scatter(list(range(len(x1))),x1, s=10, color='red', label='unknowns')
plt.scatter(list(range(len(x2))),x2, s=10, color='green', label='woman')
plt.scatter(list(range(len(x3))),x3, s=10, color='blue', label='man')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Treatment)')
plt.xlabel('Exp sum: u: ' + str(sum_x1) + ' f: ' + str(sum_x2) + ' m: ' + str(sum_x3))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('All.png')
upload_file('All.png', amazon_disptr_folder)
plt.close()

### DATE ###

plt.scatter(list(range(len(x1_date))),x1_date, s=10, color='red', label='unknowns')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Treatment)')
plt.xlabel('Exp sum: ' + str(sum_x1_date))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('0-unknowns-date.png')
upload_file('0-unknowns-date.png', date_disptr_folder)
plt.close()

plt.scatter(list(range(len(x2_date))),x2_date, s=10, color='green', label='woman')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Treatment)')
plt.xlabel('Exp sum: ' + str(sum_x2_date))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('1-woman-date.png')
upload_file('1-woman-date.png', date_disptr_folder)
plt.close()

plt.scatter(list(range(len(x3_date))),x3_date, s=10, color='blue', label='man')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Treatment)')
plt.xlabel('Exp sum: ' + str(sum_x3_date))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('2-man-date.png')
upload_file('2-man-date.png', date_disptr_folder)
plt.close()

plt.scatter(list(range(len(x1_date))),x1_date, s=10, color='red', label='unknowns')
plt.scatter(list(range(len(x2_date))),x2_date, s=10, color='green', label='woman')
plt.scatter(list(range(len(x3_date))),x3_date, s=10, color='blue', label='man')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Treatment)')
plt.xlabel('Exp sum: u: ' + str(sum_x1_date) + ' f: ' + str(sum_x2_date) + ' m: ' + str(sum_x3_date))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('All-date.png')
upload_file('All-date.png', date_disptr_folder)
plt.close()

### RANDOM ###

plt.scatter(list(range(len(x1_random))),x1_random, s=10, color='red', label='unknowns')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Treatment)')
plt.xlabel('Exp sum: ' + str(sum_x1_random))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('0-unknowns-random.png')
upload_file('0-unknowns-random.png', rand_disptr_folder)
plt.close()

plt.scatter(list(range(len(x2_random))),x2_random, s=10, color='green', label='woman')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Treatment)')
plt.xlabel('Exp sum: ' + str(sum_x2_random))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('1-woman-random.png')
upload_file('1-woman-random.png', rand_disptr_folder)
plt.close()

plt.scatter(list(range(len(x3_random))),x3_random, s=10, color='blue', label='man')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Treatment)')
plt.xlabel('Exp sum: ' + str(sum_x3_random))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('2-man-random.png')
upload_file('2-man-random.png', rand_disptr_folder)
plt.close()

plt.scatter(list(range(len(x1_random))),x1_random, s=10, color='red', label='unknowns')
plt.scatter(list(range(len(x2_random))),x2_random, s=10, color='green', label='woman')
plt.scatter(list(range(len(x3_random))),x3_random, s=10, color='blue', label='man')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Treatment)')
plt.xlabel('Exp sum: u: ' + str(sum_x1_random) + ' f: ' + str(sum_x2_random) + ' m: ' + str(sum_x3_random))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('All-random.png')
upload_file('All-random.png', rand_disptr_folder)
plt.close()

### ALL ###

min_lenght = len(x1)
if len(x1_date) < min_lenght:
  min_lenght = len(x1_date)
if len(x1_random) < min_lenght:
  min_lenght = len(x1_random)
if len(x2) < min_lenght:
  min_lenght = len(x2)
if len(x2_date) < min_lenght:
  min_lenght = len(x2_date)
if len(x2_random) < min_lenght:
  min_lenght = len(x2_random)
if len(x3) < min_lenght:
  min_lenght = len(x3)
if len(x3_date) < min_lenght:
  min_lenght = len(x3_date)
if len(x3_random) < min_lenght:
  min_lenght = len(x3_random)
print("### PRE ALL ###")

x1 = x1[:min_lenght]
x1_date = x1_date[:min_lenght]
x1_random = x1_random[:min_lenght]
x2 = x2[:min_lenght]
x2_date = x2_date[:min_lenght]
x2_random = x2_random[:min_lenght]
x3 = x3[:min_lenght]
x3_date = x3_date[:min_lenght]
x3_random = x3_random[:min_lenght]

sum_x1 = sum(x1)
sum_x1_date = sum(x1_date)
sum_x1_random = sum(x1_random)
sum_x2 = sum(x2)
sum_x2_date = sum(x2_date)
sum_x2_random = sum(x2_random)
sum_x3 = sum(x3)
sum_x3_date = sum(x3_date)
sum_x3_random = sum(x3_random)

print("### ALL ###")

plt.scatter(list(range(len(x1))),x1, s=10, color='red', label='amazon')
plt.scatter(list(range(len(x1_date))),x1_date, s=10, color='green', label='date')
plt.scatter(list(range(len(x1_random))),x1_random, s=10, color='blue', label='random')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Treatment)')
plt.xlabel('Exp sum: amazon: ' + str(sum_x1) + ' date: ' + str(sum_x1_date) + ' rand: ' + str(sum_x1_random))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('0-unknowns-allranks.png')
upload_file('0-unknowns-allranks.png', all_disptr_folder)
plt.close()

plt.scatter(list(range(len(x2))),x2, s=10, color='red', label='amazon')
plt.scatter(list(range(len(x2_date))),x2_date, s=10, color='green', label='date')
plt.scatter(list(range(len(x2_random))),x2_random, s=10, color='blue', label='random')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Treatment)')
plt.xlabel('Exp sum: amazon: ' + str(sum_x2) + ' date: ' + str(sum_x2_date) + ' rand: ' + str(sum_x2_random))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('1-woman-allranks.png')
upload_file('1-woman-allranks.png', all_disptr_folder)
plt.close()

plt.scatter(list(range(len(x3))),x3, s=10, color='red', label='amazon')
plt.scatter(list(range(len(x3_date))),x3_date, s=10, color='green', label='date')
plt.scatter(list(range(len(x3_random))),x3_random, s=10, color='blue', label='random')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Treatment)')
plt.xlabel('Exp sum: amazon: ' + str(sum_x3) + ' date: ' + str(sum_x3_date) + ' rand: ' + str(sum_x3_random))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('2-man-allranks.png')
upload_file('2-man-allranks.png', all_disptr_folder)
plt.close()

In [ ]:
#GENERA GRAFICI DI DISTRIBUZIONE CON TAGLI PER DEMOGRAPHIC PARITY
def taglia(lista,percentuale):
  temp = []
  point = None
  for elem in lista:
    if point is None:
      point = elem
      temp.append(point)
    if point is not None:
      if ((point-elem)/point) > percentuale:
        print(((point-elem)/point))
        point = elem
        temp.append(point)
    
  return temp

df = pd.read_csv('user_exp_demgr_custom_yelp_B01LXY19XD.csv')
df_date = pd.read_csv('user_exp_demgr_custom_date_B01LXY19XD.csv')
df_random = pd.read_csv('user_exp_demgr_custom_rand_B01LXY19XD.csv')

percentuale_taglio = 0.01

width = 0.20
y_min = 0.0
y_max = 1.5
ticks = 0.1

x1=[]
x2=[]
x3=[]

x1_date=[]
x2_date=[]
x3_date=[]

x1_random=[]
x2_random=[]
x3_random=[]

for i, el in df.iterrows():
  if el['group_id'] == 0:
    x1.append(el['exposure'])

for i, el in df.iterrows():
  if el['group_id'] == 1:
    x2.append(el['exposure'])

for i, el in df.iterrows():
  if el['group_id'] == 2:
    x3.append(el['exposure'])
    
for i, el in df_date.iterrows():
  if el['group_id'] == 0:
    x1_date.append(el['exposure'])

for i, el in df_date.iterrows():
  if el['group_id'] == 1:
    x2_date.append(el['exposure'])

for i, el in df_date.iterrows():
  if el['group_id'] == 2:
    x3_date.append(el['exposure'])

for i, el in df_random.iterrows():
  if el['group_id'] == 0:
    x1_random.append(el['exposure'])

for i, el in df_random.iterrows():
  if el['group_id'] == 1:
    x2_random.append(el['exposure'])

for i, el in df_random.iterrows():
  if el['group_id'] == 2:
    x3_random.append(el['exposure'])

x1.sort(reverse=True)
x2.sort(reverse=True)
x3.sort(reverse=True)

x1_date.sort(reverse=True)
x2_date.sort(reverse=True)
x3_date.sort(reverse=True)

x1_random.sort(reverse=True)
x2_random.sort(reverse=True)
x3_random.sort(reverse=True)


if percentuale_taglio is not 0:
  x1 = taglia(x1,percentuale_taglio)
  x2 = taglia(x2,percentuale_taglio)
  x3 = taglia(x3,percentuale_taglio)

  x1_date = taglia(x1_date,percentuale_taglio)
  x2_date = taglia(x2_date,percentuale_taglio)
  x3_date = taglia(x3_date,percentuale_taglio)

  x1_random = taglia(x1_random,percentuale_taglio)
  x2_random = taglia(x2_random,percentuale_taglio)
  x3_random = taglia(x3_random,percentuale_taglio)

min_lenght = len(x1)
if len(x2) < min_lenght:
  min_lenght = len(x2)
if len(x3) < min_lenght:
  min_lenght = len(x3)

x1 = x1[:min_lenght]
x2 = x2[:min_lenght]
x3 = x3[:min_lenght]

sum_x1 = sum(x1)
sum_x2 = sum(x2)
sum_x3 = sum(x3)
sum_x1_date = sum(x1_date)
sum_x2_date = sum(x2_date)
sum_x3_date = sum(x3_date)
sum_x1_random = sum(x1_random)
sum_x2_random = sum(x2_random)
sum_x3_random = sum(x3_random)

print(len(x1))
print(len(x2))
print(len(x3))

### AMAZON ###
generate_all_plots = 1
if generate_all_plots is not 0:
  plt.scatter(list(range(len(x1))),x1, s=10, color='red', label='unknowns')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Demographic Parity)')
  plt.xlabel('Exp sum: ' + str(sum_x1))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, ticks))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('0-unknowns.png')
  plt.close()

  plt.scatter(list(range(len(x2))),x2, s=10, color='green', label='woman')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Demographic Parity)')
  plt.xlabel('Exp sum: ' + str(sum_x2))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, ticks))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('1-woman.png')
  plt.close()

  plt.scatter(list(range(len(x3))),x3, s=10, color='blue', label='man')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Demographic Parity)')
  plt.xlabel('Exp sum: ' + str(sum_x3))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, ticks))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('2-man.png')
  plt.close()

  plt.scatter(list(range(len(x1))),x1, s=10, color='red', label='unknowns')
  plt.scatter(list(range(len(x2))),x2, s=10, color='green', label='woman')
  plt.scatter(list(range(len(x3))),x3, s=10, color='blue', label='man')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Demographic Parity)')
  plt.xlabel('Exp sum: u: ' + str(sum_x1) + ' f: ' + str(sum_x2) + ' m: ' + str(sum_x3))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, ticks))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('All.png')
  plt.close()

  ### DATE ###

  plt.scatter(list(range(len(x1_date))),x1_date, s=10, color='red', label='unknowns')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Demographic Parity)')
  plt.xlabel('Exp sum: ' + str(sum_x1_date))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, ticks))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('0-unknowns-date.png')
  plt.close()

  plt.scatter(list(range(len(x2_date))),x2_date, s=10, color='green', label='woman')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Demographic Parity)')
  plt.xlabel('Exp sum: ' + str(sum_x2_date))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, ticks))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('1-woman-date.png')
  plt.close()

  plt.scatter(list(range(len(x3_date))),x3_date, s=10, color='blue', label='man')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Demographic Parity)')
  plt.xlabel('Exp sum: ' + str(sum_x3_date))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, ticks))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('2-man-date.png')
  plt.close()

  plt.scatter(list(range(len(x1_date))),x1_date, s=10, color='red', label='unknowns')
  plt.scatter(list(range(len(x2_date))),x2_date, s=10, color='green', label='woman')
  plt.scatter(list(range(len(x3_date))),x3_date, s=10, color='blue', label='man')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Demographic Parity)')
  plt.xlabel('Exp sum: u: ' + str(sum_x1_date) + ' f: ' + str(sum_x2_date) + ' m: ' + str(sum_x3_date))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, ticks))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('All-date.png')
  plt.close()

  ### RANDOM ###

  plt.scatter(list(range(len(x1_random))),x1_random, s=10, color='red', label='unknowns')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Demographic Parity)')
  plt.xlabel('Exp sum: ' + str(sum_x1_random))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, ticks))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('0-unknowns-random.png')
  plt.close()

  plt.scatter(list(range(len(x2_random))),x2_random, s=10, color='green', label='woman')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Demographic Parity)')
  plt.xlabel('Exp sum: ' + str(sum_x2_random))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, ticks))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('1-woman-random.png')
  plt.close()

  plt.scatter(list(range(len(x3_random))),x3_random, s=10, color='blue', label='man')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Demographic Parity)')
  plt.xlabel('Exp sum: ' + str(sum_x3_random))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, ticks))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('2-man-random.png')
  plt.close()

  plt.scatter(list(range(len(x1_random))),x1_random, s=10, color='red', label='unknowns')
  plt.scatter(list(range(len(x2_random))),x2_random, s=10, color='green', label='woman')
  plt.scatter(list(range(len(x3_random))),x3_random, s=10, color='blue', label='man')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Demographic Parity)')
  plt.xlabel('Exp sum: u: ' + str(sum_x1_random) + ' f: ' + str(sum_x2_random) + ' m: ' + str(sum_x3_random))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, ticks))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('All-random.png')
  plt.close()

plt.scatter(list(range(len(x1))),x1, s=10, color='red', label='amazon')
plt.scatter(list(range(len(x1_date))),x1_date, s=10, color='green', label='date')
plt.scatter(list(range(len(x1_random))),x1_random, s=10, color='blue', label='random')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Demographic Parity)')
plt.xlabel('Exp sum: amazon: ' + str(sum_x1) + ' date: ' + str(sum_x1_date) + ' rand: ' + str(sum_x1_random))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('0-unknowns-allranks.png')
plt.close()

plt.scatter(list(range(len(x2))),x2, s=10, color='red', label='amazon')
plt.scatter(list(range(len(x2_date))),x2_date, s=10, color='green', label='date')
plt.scatter(list(range(len(x2_random))),x2_random, s=10, color='blue', label='random')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Demographic Parity)')
plt.xlabel('Exp sum: amazon: ' + str(sum_x2) + ' date: ' + str(sum_x2_date) + ' rand: ' + str(sum_x2_random))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('1-woman-allranks.png')
plt.close()

plt.scatter(list(range(len(x3))),x3, s=10, color='red', label='amazon')
plt.scatter(list(range(len(x3_date))),x3_date, s=10, color='green', label='date')
plt.scatter(list(range(len(x3_random))),x3_random, s=10, color='blue', label='random')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Demographic Parity)')
plt.xlabel('Exp sum: amazon: ' + str(sum_x3) + ' date: ' + str(sum_x3_date) + ' rand: ' + str(sum_x3_random))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('2-man-allranks.png')
plt.close()

In [ ]:
#GENERA GRAFICI DI DISTRIBUZIONE CON TAGLI PER DISPARATE IMPACT
def taglia(lista,percentuale):
  temp = []
  point = None
  for elem in lista:
    if point is None:
      point = elem
      temp.append(point)
    if point is not None:
      if ((point-elem)/point) > percentuale:
        print(((point-elem)/point))
        point = elem
        temp.append(point)
    
  return temp

df = pd.read_csv('user_exp_dispimp_custom_yelp_B01LXY19XD.csv')
df_date = pd.read_csv('user_exp_dispimp_custom_date_B01LXY19XD.csv')
df_random = pd.read_csv('user_exp_dispimp_custom_rand_B01LXY19XD.csv')

percentuale_taglio = 0.01

width = 0.20
y_min = 0.0
y_max = 12

x1=[]
x2=[]
x3=[]

x1_date=[]
x2_date=[]
x3_date=[]

x1_random=[]
x2_random=[]
x3_random=[]

for i, el in df.iterrows():
  if el['group_id'] == 0:
    x1.append(el['exposure'])

for i, el in df.iterrows():
  if el['group_id'] == 1:
    x2.append(el['exposure'])

for i, el in df.iterrows():
  if el['group_id'] == 2:
    x3.append(el['exposure'])
    
for i, el in df_date.iterrows():
  if el['group_id'] == 0:
    x1_date.append(el['exposure'])

for i, el in df_date.iterrows():
  if el['group_id'] == 1:
    x2_date.append(el['exposure'])

for i, el in df_date.iterrows():
  if el['group_id'] == 2:
    x3_date.append(el['exposure'])

for i, el in df_random.iterrows():
  if el['group_id'] == 0:
    x1_random.append(el['exposure'])

for i, el in df_random.iterrows():
  if el['group_id'] == 1:
    x2_random.append(el['exposure'])

for i, el in df_random.iterrows():
  if el['group_id'] == 2:
    x3_random.append(el['exposure'])

sum_x1 = sum(x1)
sum_x2 = sum(x2)
sum_x3 = sum(x3)
sum_x1_date = sum(x1_date)
sum_x2_date = sum(x2_date)
sum_x3_date = sum(x3_date)
sum_x1_random = sum(x1_random)
sum_x2_random = sum(x2_random)
sum_x3_random = sum(x3_random)

x1.sort(reverse=True)
x2.sort(reverse=True)
x3.sort(reverse=True)

x1_date.sort(reverse=True)
x2_date.sort(reverse=True)
x3_date.sort(reverse=True)

x1_random.sort(reverse=True)
x2_random.sort(reverse=True)
x3_random.sort(reverse=True)


if percentuale_taglio is not 0:

  x1 = taglia(x1,percentuale_taglio)
  x2 = taglia(x2,percentuale_taglio)
  x3 = taglia(x3,percentuale_taglio)


  x1_date = taglia(x1_date,percentuale_taglio)
  x2_date = taglia(x2_date,percentuale_taglio)
  x3_date = taglia(x3_date,percentuale_taglio)

  x1_random = taglia(x1_random,percentuale_taglio)
  x2_random = taglia(x2_random,percentuale_taglio)
  x3_random = taglia(x3_random,percentuale_taglio)

### AMAZON ###
generate_all_plots = 1
if generate_all_plots is not 0:
  plt.scatter(list(range(len(x1))),x1, s=10, color='red', label='unknowns')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Disparate Impact)')
  plt.xlabel('Exp sum: ' + str(sum_x1))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, 1))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('0-unknowns.png')
  plt.close()

  plt.scatter(list(range(len(x2))),x2, s=10, color='green', label='woman')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Disparate Impact)')
  plt.xlabel('Exp sum: ' + str(sum_x2))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, 1))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('1-woman.png')
  plt.close()

  plt.scatter(list(range(len(x3))),x3, s=10, color='blue', label='man')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Disparate Impact)')
  plt.xlabel('Exp sum: ' + str(sum_x3))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, 1))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('2-man.png')
  plt.close()

  plt.scatter(list(range(len(x1))),x1, s=10, color='red', label='unknowns')
  plt.scatter(list(range(len(x2))),x2, s=10, color='green', label='woman')
  plt.scatter(list(range(len(x3))),x3, s=10, color='blue', label='man')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Disparate Impact)')
  plt.xlabel('Exp sum: u: ' + str(sum_x1) + ' f: ' + str(sum_x2) + ' m: ' + str(sum_x3))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, 1))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('All.png')
  plt.close()

  ### DATE ###

  plt.scatter(list(range(len(x1_date))),x1_date, s=10, color='red', label='unknowns')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Disparate Impact)')
  plt.xlabel('Exp sum: ' + str(sum_x1_date))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, 1))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('0-unknowns-date.png')
  plt.close()

  plt.scatter(list(range(len(x2_date))),x2_date, s=10, color='green', label='woman')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Disparate Impact)')
  plt.xlabel('Exp sum: ' + str(sum_x2_date))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, 1))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('1-woman-date.png')
  plt.close()

  plt.scatter(list(range(len(x3_date))),x3_date, s=10, color='blue', label='man')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Disparate Impact)')
  plt.xlabel('Exp sum: ' + str(sum_x3_date))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, 1))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('2-man-date.png')
  plt.close()

  plt.scatter(list(range(len(x1_date))),x1_date, s=10, color='red', label='unknowns')
  plt.scatter(list(range(len(x2_date))),x2_date, s=10, color='green', label='woman')
  plt.scatter(list(range(len(x3_date))),x3_date, s=10, color='blue', label='man')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Disparate Impact)')
  plt.xlabel('Exp sum: u: ' + str(sum_x1_date) + ' f: ' + str(sum_x2_date) + ' m: ' + str(sum_x3_date))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, 1))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('All-date.png')
  plt.close()

  ### RANDOM ###

  plt.scatter(list(range(len(x1_random))),x1_random, s=10, color='red', label='unknowns')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Disparate Impact)')
  plt.xlabel('Exp sum: ' + str(sum_x1_random))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, 1))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('0-unknowns-random.png')
  plt.close()

  plt.scatter(list(range(len(x2_random))),x2_random, s=10, color='green', label='woman')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Disparate Impact)')
  plt.xlabel('Exp sum: ' + str(sum_x2_random))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, 1))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('1-woman-random.png')
  plt.close()

  plt.scatter(list(range(len(x3_random))),x3_random, s=10, color='blue', label='man')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Disparate Impact)')
  plt.xlabel('Exp sum: ' + str(sum_x3_random))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, 1))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('2-man-random.png')
  plt.close()

  plt.scatter(list(range(len(x1_random))),x1_random, s=10, color='red', label='unknowns')
  plt.scatter(list(range(len(x2_random))),x2_random, s=10, color='green', label='woman')
  plt.scatter(list(range(len(x3_random))),x3_random, s=10, color='blue', label='man')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Disparate Impact)')
  plt.xlabel('Exp sum: u: ' + str(sum_x1_random) + ' f: ' + str(sum_x2_random) + ' m: ' + str(sum_x3_random))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, 1))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('All-random.png')
  plt.close()

plt.scatter(list(range(len(x1))),x1, s=10, color='red', label='amazon')
plt.scatter(list(range(len(x1_date))),x1_date, s=10, color='green', label='date')
plt.scatter(list(range(len(x1_random))),x1_random, s=10, color='blue', label='random')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Impact)')
plt.xlabel('Exp sum: amazon: ' + str(sum_x1) + ' date: ' + str(sum_x1_date) + ' rand: ' + str(sum_x1_random))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, 1))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('0-unknowns-allranks.png')
plt.close()

plt.scatter(list(range(len(x2))),x2, s=10, color='red', label='amazon')
plt.scatter(list(range(len(x2_date))),x2_date, s=10, color='green', label='date')
plt.scatter(list(range(len(x2_random))),x2_random, s=10, color='blue', label='random')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Impact)')
plt.xlabel('Exp sum: amazon: ' + str(sum_x2) + ' date: ' + str(sum_x2_date) + ' rand: ' + str(sum_x2_random))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, 1))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('1-woman-allranks.png')
plt.close()

plt.scatter(list(range(len(x3))),x3, s=10, color='red', label='amazon')
plt.scatter(list(range(len(x3_date))),x3_date, s=10, color='green', label='date')
plt.scatter(list(range(len(x3_random))),x3_random, s=10, color='blue', label='random')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Impact)')
plt.xlabel('Exp sum: amazon: ' + str(sum_x3) + ' date: ' + str(sum_x3_date) + ' rand: ' + str(sum_x3_random))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, 1))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('2-man-allranks.png')
plt.close()

In [ ]:
#GENERA GRAFICI DI DISTRIBUZIONE CON TAGLI PER DISPARATE TREATMENT
def taglia(lista,percentuale):
  temp = []
  point = None
  for elem in lista:
    if point is None:
      point = elem
      temp.append(point)
    if point is not None:
      if ((point-elem)/point) > percentuale:
        print(((point-elem)/point))
        point = elem
        temp.append(point)
    
  return temp

df = pd.read_csv('user_exp_disptr_custom_yelp_B01LXY19XD.csv')
df_date = pd.read_csv('user_exp_disptr_custom_date_B01LXY19XD.csv')
df_random = pd.read_csv('user_exp_disptr_custom_rand_B01LXY19XD.csv')

percentuale_taglio = 0.02

width = 0.20
y_min = 0.0
y_max = 1.2
ticks = 0.1

x1=[]
x2=[]
x3=[]

x1_date=[]
x2_date=[]
x3_date=[]

x1_random=[]
x2_random=[]
x3_random=[]

for i, el in df.iterrows():
  if el['group_id'] == 0:
    x1.append(el['exposure'])

for i, el in df.iterrows():
  if el['group_id'] == 1:
    x2.append(el['exposure'])

for i, el in df.iterrows():
  if el['group_id'] == 2:
    x3.append(el['exposure'])
    
for i, el in df_date.iterrows():
  if el['group_id'] == 0:
    x1_date.append(el['exposure'])

for i, el in df_date.iterrows():
  if el['group_id'] == 1:
    x2_date.append(el['exposure'])

for i, el in df_date.iterrows():
  if el['group_id'] == 2:
    x3_date.append(el['exposure'])

for i, el in df_random.iterrows():
  if el['group_id'] == 0:
    x1_random.append(el['exposure'])

for i, el in df_random.iterrows():
  if el['group_id'] == 1:
    x2_random.append(el['exposure'])

for i, el in df_random.iterrows():
  if el['group_id'] == 2:
    x3_random.append(el['exposure'])

sum_x1 = sum(x1)
sum_x2 = sum(x2)
sum_x3 = sum(x3)
sum_x1_date = sum(x1_date)
sum_x2_date = sum(x2_date)
sum_x3_date = sum(x3_date)
sum_x1_random = sum(x1_random)
sum_x2_random = sum(x2_random)
sum_x3_random = sum(x3_random)

x1.sort(reverse=True)
x2.sort(reverse=True)
x3.sort(reverse=True)

x1_date.sort(reverse=True)
x2_date.sort(reverse=True)
x3_date.sort(reverse=True)

x1_random.sort(reverse=True)
x2_random.sort(reverse=True)
x3_random.sort(reverse=True)


if percentuale_taglio is not 0:
  x1 = taglia(x1,percentuale_taglio)
  x2 = taglia(x2,percentuale_taglio)
  x3 = taglia(x3,percentuale_taglio)

  x1_date = taglia(x1_date,percentuale_taglio)
  x2_date = taglia(x2_date,percentuale_taglio)
  x3_date = taglia(x3_date,percentuale_taglio)

  x1_random = taglia(x1_random,percentuale_taglio)
  x2_random = taglia(x2_random,percentuale_taglio)
  x3_random = taglia(x3_random,percentuale_taglio)

print(x1)
print(x2)
print(x3)

### AMAZON ###
generate_all_plots = 1
if generate_all_plots is not 0:
  plt.scatter(list(range(len(x1))),x1, s=10, color='red', label='unknowns')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Disparate Treatment)')
  plt.xlabel('Exp sum: ' + str(sum_x1))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, ticks))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('0-unknowns.png')
  plt.close()

  plt.scatter(list(range(len(x2))),x2, s=10, color='green', label='woman')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Disparate Treatment)')
  plt.xlabel('Exp sum: ' + str(sum_x2))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, ticks))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('1-woman.png')
  plt.close()

  plt.scatter(list(range(len(x3))),x3, s=10, color='blue', label='man')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Disparate Treatment)')
  plt.xlabel('Exp sum: ' + str(sum_x3))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, ticks))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('2-man.png')
  plt.close()

  plt.scatter(list(range(len(x1))),x1, s=10, color='red', label='unknowns')
  plt.scatter(list(range(len(x2))),x2, s=10, color='green', label='woman')
  plt.scatter(list(range(len(x3))),x3, s=10, color='blue', label='man')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Disparate Treatment)')
  plt.xlabel('Exp sum: u: ' + str(sum_x1) + ' f: ' + str(sum_x2) + ' m: ' + str(sum_x3))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, ticks))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('All.png')
  plt.close()

  ### DATE ###

  plt.scatter(list(range(len(x1_date))),x1_date, s=10, color='red', label='unknowns')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Disparate Treatment)')
  plt.xlabel('Exp sum: ' + str(sum_x1_date))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, ticks))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('0-unknowns-date.png')
  plt.close()

  plt.scatter(list(range(len(x2_date))),x2_date, s=10, color='green', label='woman')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Disparate Treatment)')
  plt.xlabel('Exp sum: ' + str(sum_x2_date))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, ticks))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('1-woman-date.png')
  plt.close()

  plt.scatter(list(range(len(x3_date))),x3_date, s=10, color='blue', label='man')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Disparate Treatment)')
  plt.xlabel('Exp sum: ' + str(sum_x3_date))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, ticks))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('2-man-date.png')
  plt.close()

  plt.scatter(list(range(len(x1_date))),x1_date, s=10, color='red', label='unknowns')
  plt.scatter(list(range(len(x2_date))),x2_date, s=10, color='green', label='woman')
  plt.scatter(list(range(len(x3_date))),x3_date, s=10, color='blue', label='man')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Disparate Treatment)')
  plt.xlabel('Exp sum: u: ' + str(sum_x1_date) + ' f: ' + str(sum_x2_date) + ' m: ' + str(sum_x3_date))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, ticks))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('All-date.png')
  plt.close()

  ### RANDOM ###

  plt.scatter(list(range(len(x1_random))),x1_random, s=10, color='red', label='unknowns')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Disparate Treatment)')
  plt.xlabel('Exp sum: ' + str(sum_x1_random))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, ticks))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('0-unknowns-random.png')
  plt.close()

  plt.scatter(list(range(len(x2_random))),x2_random, s=10, color='green', label='woman')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Disparate Treatment)')
  plt.xlabel('Exp sum: ' + str(sum_x2_random))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, ticks))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('1-woman-random.png')
  plt.close()

  plt.scatter(list(range(len(x3_random))),x3_random, s=10, color='blue', label='man')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Disparate Treatment)')
  plt.xlabel('Exp sum: ' + str(sum_x3_random))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, ticks))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('2-man-random.png')
  plt.close()

  plt.scatter(list(range(len(x1_random))),x1_random, s=10, color='red', label='unknowns')
  plt.scatter(list(range(len(x2_random))),x2_random, s=10, color='green', label='woman')
  plt.scatter(list(range(len(x3_random))),x3_random, s=10, color='blue', label='man')
  plt.legend(loc="upper center")
  plt.ylabel('Exposure (Disparate Treatment)')
  plt.xlabel('Exp sum: u: ' + str(sum_x1_random) + ' f: ' + str(sum_x2_random) + ' m: ' + str(sum_x3_random))
  plt.xticks([])
  plt.yticks(np.arange(y_min, y_max, ticks))
  axes = plt.gca()
  axes.set_ylim([y_min,y_max])
  axes.yaxis.grid()
  plt.savefig('All-random.png')
  plt.close()

plt.scatter(list(range(len(x1))),x1, s=10, color='red', label='amazon')
plt.scatter(list(range(len(x1_date))),x1_date, s=10, color='green', label='date')
plt.scatter(list(range(len(x1_random))),x1_random, s=10, color='blue', label='random')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Treatment)')
plt.xlabel('Exp sum: amazon: ' + str(sum_x1) + ' date: ' + str(sum_x1_date) + ' rand: ' + str(sum_x1_random))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('0-unknowns-allranks.png')
plt.close()

plt.scatter(list(range(len(x2))),x2, s=10, color='red', label='amazon')
plt.scatter(list(range(len(x2_date))),x2_date, s=10, color='green', label='date')
plt.scatter(list(range(len(x2_random))),x2_random, s=10, color='blue', label='random')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Treatment)')
plt.xlabel('Exp sum: amazon: ' + str(sum_x2) + ' date: ' + str(sum_x2_date) + ' rand: ' + str(sum_x2_random))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('1-woman-allranks.png')
plt.close()

plt.scatter(list(range(len(x3))),x3, s=10, color='red', label='amazon')
plt.scatter(list(range(len(x3_date))),x3_date, s=10, color='green', label='date')
plt.scatter(list(range(len(x3_random))),x3_random, s=10, color='blue', label='random')
plt.legend(loc="upper center")
plt.ylabel('Exposure (Disparate Treatment)')
plt.xlabel('Exp sum: amazon: ' + str(sum_x3) + ' date: ' + str(sum_x3_date) + ' rand: ' + str(sum_x3_random))
plt.xticks([])
plt.yticks(np.arange(y_min, y_max, ticks))
axes = plt.gca()
axes.set_ylim([y_min,y_max])
axes.yaxis.grid()
plt.savefig('2-man-allranks.png')
plt.close()

In [ ]:
#ESECUZIONE VELOCE SENZA ASSUNZIONI; MULTI-RANKING INCLUSO
drive = authenticate()
download = drive.CreateFile({'id': '1V-L7IqaPVcBp4RvWKnUXt00SHrsT1e9O'})
download.GetContentFile('queries.csv')

queries = pd.read_csv("queries.csv")
queries = drop_unnamed(queries)

query=4
products_gtree_id = queries.loc[queries["cardinal"]==query,"products_gtree"].tolist()[0]
products_folder_id = queries.loc[queries["cardinal"]==query,"products_folder"].tolist()[0]
reviews_folder_id = queries.loc[queries["cardinal"]==query,"reviews_folder"].tolist()[0]
fairness_data_id = queries.loc[queries["cardinal"]==query,"fairness_data"].tolist()[0]
face_data_id = queries.loc[queries["cardinal"]==query,"face_data"].tolist()[0]

download = drive.CreateFile({'id': products_gtree_id})
download.GetContentFile(str(query)+'_gtree.csv')

gtree = pd.read_csv(str(query)+'_gtree.csv')
gtree = drop_unnamed(gtree)
#gtree['asin'].tolist()
products_id_list = ["B01LXY19XD"]

#list_of_attributes = ['review_count'] # alluser
#list_of_attributes = ['review_count', 'fans', 'average_stars', 'useful', 'funny', 'cool'] # dataset
#list_of_attributes = ['loc1', 'loc2', 'loc3'] # dataset
#list_of_attributes = ['age', 'gender', 'ethnicity'] # alluser
#list_of_attributes = ['fans'] # dataset
#list_of_attributes = ['useful', 'funny', 'cool'] # dataset
#list_of_attributes = ['gender', 'ethnicity'] # alluser
#list_of_attributes = ['ethnicity'] # alluser
#list_of_attributes = ['review_count', 'friend_count', 'photo_count'] # alluser
#list_of_attributes = ['review_sentiment'] # alluser
list_of_attributes = ['gender'] # alluser
method = 'custom'
folder_name = '#14_custom_gender_'
gtree_folder_destination_name = 'gfolder_#14'
N_of_groups = 3
alluser = True

for id in products_id_list:
  print(id)
  drive = authenticate()
  df_rel_ranking, df_date_ranking, df_rand_ranking = pipeline1(id, alluser) 
  
  destination = create_folder_in_drive(drive, folder_name+str(id), products_folder_id)
  destination_copy = destination

  group_list, percents, destination = pipeline2(df_rel_ranking, id,
                                  N_of_groups, list_of_attributes, method,
                                  alluser, destination, '')
  
  pipeline3(id, method, df_rel_ranking, df_date_ranking, df_rand_ranking,
            percents, list_of_attributes, alluser, destination, group_list)
      # MULTI RANKING
  drive = authenticate()
  print('''#####################################################################
            ################## MULTI RANKING ####################################
            #####################################################################''')
  position = gtree.loc[gtree['asin']==id,"position"].tolist()[0]
  votes = gtree['vote'].tolist()
  reviewers = gtree['reviews_count'].tolist()
  
  destination_new = create_folder_in_drive(drive, "multi_rank", destination)

  group_list, percents, destination = pipeline2(df_rel_ranking, id,
                                  N_of_groups, list_of_attributes, method,
                                  alluser, destination_new, '')
  
  print(percents)

  pipeline3m(id, method, df_rel_ranking, df_date_ranking, df_rand_ranking,
            percents, list_of_attributes, alluser, destination, group_list, position, votes, reviewers)
print('FIN')

In [ ]:
#ESECUZIONE CON ASSUNZIONI
drive = authenticate()
download = drive.CreateFile({'id': '1V-L7IqaPVcBp4RvWKnUXt00SHrsT1e9O'})
download.GetContentFile('queries.csv')

queries = pd.read_csv("queries.csv")
queries = drop_unnamed(queries)

query=4
products_gtree_id = queries.loc[queries["cardinal"]==query,"products_gtree"].tolist()[0]
products_folder_id = queries.loc[queries["cardinal"]==query,"products_folder"].tolist()[0]
reviews_folder_id = queries.loc[queries["cardinal"]==query,"reviews_folder"].tolist()[0]
fairness_data_id = queries.loc[queries["cardinal"]==query,"fairness_data"].tolist()[0]
face_data_id = queries.loc[queries["cardinal"]==query,"face_data"].tolist()[0]

download = drive.CreateFile({'id': products_gtree_id})
download.GetContentFile(str(query)+'_gtree.csv')

gtree = pd.read_csv(str(query)+'_gtree.csv')
gtree = drop_unnamed(gtree)

products_id_list = ["B01LXY19XD"]

#list_of_attributes = ['review_count'] # alluser
#list_of_attributes = ['review_count', 'fans', 'average_stars', 'useful', 'funny', 'cool'] # dataset
#list_of_attributes = ['loc1', 'loc2', 'loc3'] # dataset
#list_of_attributes = ['age', 'gender', 'ethnicity'] # alluser
#list_of_attributes = ['fans'] # dataset
#list_of_attributes = ['useful', 'funny', 'cool'] # dataset
#list_of_attributes = ['gender', 'ethnicity'] # alluser
#list_of_attributes = ['ethnicity'] # alluser
#list_of_attributes = ['review_count', 'friend_count', 'photo_count'] # alluser
#list_of_attributes = ['review_sentiment'] # alluser
list_of_attributes = ['gender'] # alluser
method = 'custom'
folder_name = '#14_custom_gender_'
gtree_folder_destination_name = 'gfolder_#14'
N_of_groups = 3
alluser = True

for id in products_id_list:
  try:
    print(id)
    drive = authenticate()
    df_rel_ranking, df_date_ranking, df_rand_ranking = pipeline1(id, alluser) 
    
    destination = create_folder_in_drive(drive, folder_name+str(id), products_folder_id)
    destination_copy = destination

    group_list, percents, destination = pipeline2(df_rel_ranking, id,
                                    N_of_groups, list_of_attributes, method,
                                    alluser, destination, '')
    
    pipeline3(id, method, df_rel_ranking, df_date_ranking, df_rand_ranking,
              percents, list_of_attributes, alluser, destination, group_list)
    
    # MULTI RANKING
    drive = authenticate()
    print('''#####################################################################
              ################## MULTI RANKING ####################################
              #####################################################################''')
    position = gtree.loc[gtree['asin']==id,"position"].tolist()[0]
    votes = gtree['vote'].tolist()
    reviewers = gtree['reviews_count'].tolist()
    
    destination_new = create_folder_in_drive(drive, "multi_rank", destination)

    group_list, percents, destination = pipeline2(df_rel_ranking, id,
                                    N_of_groups, list_of_attributes, method,
                                    alluser, destination_new, '')
    
    print(percents)

    pipeline3m(id, method, df_rel_ranking, df_date_ranking, df_rand_ranking,
              percents, list_of_attributes, alluser, destination, group_list, position, votes, reviewers)
    # ASSUMPTIONS
    # ALL MEN
    drive = authenticate()
    print('''#####################################################################
              ################## ASSUMPTION ALL MEN ###############################
              #####################################################################''')

    destination = destination_copy
    group_list, percents, destination = pipeline2(df_rel_ranking, id,
                                    N_of_groups, list_of_attributes, method,
                                    alluser, destination, 'all_men')
    
    print(percents)

    pipeline3(id, method, df_rel_ranking, df_date_ranking, df_rand_ranking,
              percents, list_of_attributes, alluser, destination, group_list)
    
    # ALL WOMEN
    drive = authenticate()
    print('''#####################################################################
              ################## ASSUMPTION ALL WOMEN #############################
              #####################################################################''')
    destination = destination_copy
    group_list, percents, destination = pipeline2(df_rel_ranking, id,
                                    N_of_groups, list_of_attributes, method,
                                    alluser, destination, 'all_women')
    print(group_list)
    print(percents)

    pipeline3(id, method, df_rel_ranking, df_date_ranking, df_rand_ranking,
              percents, list_of_attributes, alluser, destination, group_list)
    
    # 50%
    drive = authenticate()
    print('''#####################################################################
              ################## ASSUMPTION 50 AND 50 #############################
              #####################################################################''')
    destination = destination_copy
    group_list, percents, destination = pipeline2(df_rel_ranking, id,
                                    N_of_groups, list_of_attributes, method,
                                    alluser, destination, '50')
    print(group_list)
    print(percents)

    pipeline3(id, method, df_rel_ranking, df_date_ranking, df_rand_ranking, 
              percents, list_of_attributes, alluser, destination, group_list)

    # EQUALLY DISTRIBUTED
    drive = authenticate()
    print('''#####################################################################
              ################## ASSUMPTION EQUALLY DISTRIBUTED ###################
              #####################################################################''')
    destination = destination_copy
    group_list, percents, destination = pipeline2(df_rel_ranking, id,
                                    N_of_groups, list_of_attributes, method,
                                    alluser, destination, 'equal')
    print(group_list)
    print(percents)

    pipeline3(id, method, df_rel_ranking, df_date_ranking, df_rand_ranking,
              percents, list_of_attributes, alluser, destination, group_list)
    
    # MAINTAIN PROPORTION
    drive = authenticate()
    print('''#####################################################################
              ################## ASSUMPTION MAINTAIN PROPORTION ###################
              #####################################################################''')
    destination = destination_copy
    group_list, percents, destination = pipeline2(df_rel_ranking, id,
                                    N_of_groups, list_of_attributes, method,
                                    alluser, destination, 'proportioned')
    print(group_list)
    print(percents)

    pipeline3(id, method, df_rel_ranking, df_date_ranking, df_rand_ranking,
              percents, list_of_attributes, alluser, destination, group_list)
  except:
    failure = pd.DataFrame()
    failure.to_csv("0_failure"+str(id)+".csv")
    upload_file("0_failure"+str(id)+".csv",products_folder_id)
    print("FAILURE")
print('FIN')